# SEE 'Directions for running' below!

In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)
# NOTE: To reload a class imported as, e.g., 
# from module import class
# One must call:
#   1. import module
#   2. reload module
#   3. from module import class

import sys, os
import re
from pathlib import Path
import json
import pickle

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_datetime64_dtype, is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns, natsort_keygen
from packaging import version
import copy

import itertools

import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
import matplotlib.colors as mcolors
import matplotlib.cm as cm #e.g. for cmap=cm.jet
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from MeterPremise import MeterPremise
#-----
from AMI_SQL import AMI_SQL
from AMINonVee_SQL import AMINonVee_SQL
from AMIEndEvents_SQL import AMIEndEvents_SQL
from AMIUsgInst_SQL import AMIUsgInst_SQL
from DOVSOutages_SQL import DOVSOutages_SQL
#-----
from GenAn import GenAn
from AMINonVee import AMINonVee
from AMIEndEvents import AMIEndEvents
from AMIEDE_DEV import AMIEDE_DEV
from MECPODf import MECPODf
from MECPOAn import MECPOAn
from AMIUsgInst import AMIUsgInst
from DOVSOutages import DOVSOutages
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_sql_aids_dir())
import Utilities_sql
import TableInfos
from TableInfos import TableInfo
from SQLElement import SQLElement
from SQLElementsCollection import SQLElementsCollection
from SQLSelect import SQLSelectElement, SQLSelect
from SQLFrom import SQLFrom
from SQLWhere import SQLWhereElement, SQLWhere
from SQLJoin import SQLJoin, SQLJoinCollection
from SQLGroupBy import SQLGroupByElement, SQLGroupBy
from SQLHaving import SQLHaving
from SQLOrderBy import SQLOrderByElement, SQLOrderBy
from SQLQuery import SQLQuery
from SQLQueryGeneric import SQLQueryGeneric
#---------------------------------------------------------------------
#sys.path.insert(0, os.path.join(os.path.realpath('..'), 'Utilities'))
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
from Utilities_df import DFConstructType
import Utilities_dt
import Plot_General
import Plot_Box_sns
import Plot_Hist
import Plot_Bar
import GrubbsTest
import DataFrameSubsetSlicer
from DataFrameSubsetSlicer import DataFrameSubsetSlicer as DFSlicer

# Directions for running:

For the purposes of this demonstration, let's assume your AEP User ID is s123456, and your local Documents directory
is located at C:\Users\s123456\Documents

1. If not already done, clone the Analysis GitHub repo (https://github.aepsc.com/s346557/Analysis).
<br>- I will assume the repo was cloned into the Documents directory, i.e. I assume your local copy of the repo is located at C:\Users\s123456\Documents\Analysis (and therefore, this Jupyter notebook should be located at C:\Users\s123456\Documents\Analysis\JupyterNbs\IT_Demo.ipynb)
<br><br>

2. Create a simple text file containing your AEP passwords.
- I suggest you use the file pwd_file_template.txt in the Analysis directory (C:\Users\s123456\Documents\Analysis\pwd_file_template.txt) to create your own password file.
    - DO NOT ALTER the pwd_file_template.txt file, create a new pwd_file.txt file!
- I further suggest you name your password file pwd_file.txt and place it in the Analysis directory (C:\Users\s123456\Documents\Analysis\pwd_file.txt).
    - The Git repo is set up to ignore pwd_file.txt in the Analysis directory, so your information will not be pushed up to the repo if saved in this manner.
- NOTE: At one point, my Athena and Oracle passwords were different, which is why there is a 'Main' and 'Oracle' entry in the password file.  Likely you will put the same password for both entries.
<br><br>

3. IF NOT ALREADY DONE, run the method Utilities_config.generate_initial_config_file to initiate your config.yaml file
- I suggest you input arguments for all three parameters (aep_user_id, pwd_file_path, and local_data_dir)
    - If no aep_user_id is given, the code will attempt to determine your AEP User ID from the contents of your C:\Users directory
    - If no pwd_file_path is given, it is assumed to exist, be named pwd_file.txt, and be located in the Analysis directory (C:\Users\s123456\Documents\Analysis\pwd_file.txt)
    - If local_data_dir is not None, it should point to a directory when you plan to store any results (my personal local_data_dir is located at C:\Users\s346557\Documents\LocalData\).
        - If you are not planning to save or load any files locally, I believe this can be kept as None

# ----------------------------------------------------------------------------------------------------
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# ONLY NEED TO RUN ONCE!
So, if you have already run Utilities_config.generate_initial_config_file (and your configuration has not changed since), there is no need to run again

In [ ]:
run_config=False

In [ ]:
if run_config:
    # REPLACE VALUES BELOW WITH YOUR OWN
    aep_user_id = 's123456'
    pwd_file_path = r'C:\Users\s3123456\Documents\Analysis\pwd_file.txt'
    local_data_dir = r'C:\Users\s123456\Documents\LocalData'

    Utilities_config.generate_initial_config_file(
        aep_user_id=aep_user_id, 
        pwd_file_path=pwd_file_path, 
        local_data_dir=local_data_dir
    )

# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# ----------------------------------------------------------------------------------------------------

In [ ]:
cols_of_interest_end_dev_event = TableInfos.AMIEndEvents_TI.std_columns_of_interest
cols_of_interest_met_prem      = TableInfos.MeterPremise_TI.std_columns_of_interest

patterns_to_replace=[
    r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', 
    
    (
        (
            r'(Under Voltage)\s*'\
            r'([0-9a-zA-Z]*)?\s*'\
            r'(\([0-9a-zA-Z\s]*\))\s*'\
            r'([0-9a-zA-Z]*)?\s?'\
            r'(for meter\:?\s*)'\
            r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
            r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
            r'(Voltage out of tolerance)?'
        ), 
        'AMIEndEvents.under_voltage_match_func'
    ), 
    
    (
        (
            r'(Last Gasp\s*-\s*[0-9a-zA-Z\s]*)[\s\:,.]*'\
            r'.*'\
            r'(Fail Reason: .*)$'
        ), 
        'AMIEndEvents.last_gasp_reduce_func'
    ), 
    
    (r'(Angle out of tolerance) \[.*\]', r'\1'), 
    
    (r'(NIC Power Restore Trap Received from device).*', r'\1'), #TODO This has much info, like last gasp
    
    (
        (
            r'(Requested operation .* could not be applied to the given device type and firmware version.) '\
            r'Device, DeviceType: .*, Firmware Version: .*$'
        ), 
        r'\1'
    ), 
    
    ('meterN/A', 'meter'),
    
    (r'(Meter needs explicit time sync.) Drift: -?\d* s, (Encountered Problems:\s*.*), Meter_Time', r'\1 \2'), 
    
    (r'(Meter Program Seal mismatch for Device) \[Device ID, MAC Id\] = .*', r'\1'), 
    
    (r'Device Time: .* Failed Device Reason: (.*) Reboot Counter: .* Refresh Counter: .*', r'\1'), 
    
    (r'(Ignoring (?:Interval|Register) Read data for device as it has time in the future) .*', r'\1'), 
    
    (r'(Secure association operation failed consecutively for 1 times for [0-9a-zA-Z]{4}.) .*', r'\1'), 
    
    (r'Device, (Last Gasp State: .*), (Detector State: .*), Reboot Count: \d*', r'\1, \2'), 
    
    (r'(Detected end of voltage sag on meter).*', r'\1'), 
    
    (r'(Detected end of voltage swell on meter).*', r'\1'), 
    
    r'N/A', 
    
    (r'\s{2,}', ' ')
]

# NOTE:
This is a somewhat simple-minded example, as I am naively joining the meter_events.end_device_event table with default.meter_premise table.
<br>As we have discussed, to correctly join with meter_premise, one must determine which meters were active AT THE TIME IN QUESTION (i.e., on 2022-10-01), not those simply in default.meter_premise.
<br>Thus, to be completely correct, one should use methods contained in the MeterPremise class.
<br>However, for the purposes here, this simple-minded join is acceptable.

In [ ]:
# Build AMIEndEvents object housing data from the meter_events.end_device_event table.
# Events are collected from those meters having aep_opco='oh' on the date '2022-10-01'
ami_ede = AMIEndEvents(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args=None, 
    init_df_in_constructor=True, 
    build_sql_function=None, 
    build_sql_function_kwargs=dict(
        cols_of_interest=cols_of_interest_end_dev_event, 
        opco='oh', 
        date_range=['2022-10-01', '2022-10-01'], 
        join_mp_args=dict(
            join_with_CTE=True, 
            build_mp_kwargs=dict(cols_of_interest=cols_of_interest_met_prem), 
            join_type='LEFT', 
            list_of_columns_to_join=[
                ['serialnumber', 'mfr_devc_ser_nbr'], 
                ['aep_premise_nb', 'prem_nb']
            ]
        )
    ), 
    save_args=False
)

In [ ]:
# The SQL statement run by the above block is printed below
print(ami_ede.get_sql_statement())

In [ ]:
# Grab the pandas.DataFrame (DF) object from ami_ede
end_events_df = ami_ede.df.copy()
# Also, grab the full list of reasons found
reasons_full = sorted(end_events_df['reason'].unique().tolist())

In [ ]:
# Print some basic info about the DF
print(f'end_events_df.shape = {end_events_df.shape}')
print(f"Number of unique reasons = {end_events_df['reason'].nunique()}")
end_events_df.head()

# -------------------------

In [ ]:
# Below, when running reduce_end_event_reasons_in_df, the default behavior (placement_col=None) is to simply replace
#   the entries in the 'reason' column by their reduced versions.
# For purposes here, I will keep both the full reason column (renamed to 'reason_full') and the reduced reason column (named 'reason')

print(f'end_events_df.shape[0]                          = {end_events_df.shape[0]}')
print(f"end_events_df['serialnumber'].nunique()         = {end_events_df['serialnumber'].nunique()}")
print(f"end_events_df['enddeviceeventtypeid'].nunique() = {end_events_df['enddeviceeventtypeid'].nunique()}")
print('\n\n')

end_events_df = end_events_df.rename(columns={'reason':'reason_full'})
end_events_df = AMIEndEvents.reduce_end_event_reasons_in_df(
    df=end_events_df, 
    patterns_to_replace=patterns_to_replace, 
    reason_col='reason_full', 
    placement_col='reason'
)

print('BEFORE REASON REDUCTION')
print(f"end_events_df['reason'].nunique()               = {end_events_df['reason_full'].nunique()}")
print('AFTER REASON REDUCTION')
print(f"end_events_df['reason'].nunique()               = {end_events_df['reason'].nunique()}")

In [ ]:
# Grab the unique reasons after the reduction (rdcd=='reduced')
reasons_rdcd_1 = sorted(end_events_df['reason'].unique().tolist())

In [ ]:
reasons_rdcd_1

# -------------------------

In [ ]:
# Create a mapping of the full reasons captured in each reason
reasons_to_full_mapping = end_events_df.groupby('reason')['reason_full'].unique().to_frame()
reasons_to_full_mapping['n_full_in_reason'] = reasons_to_full_mapping['reason_full'].apply(lambda x: len(x))
reasons_to_full_mapping

In [ ]:
# e.g., all of the full reasons contained in 'Access Point has lost connectivity with FHSS 900 MHz band'
reasons_to_full_mapping.loc['Access Point has lost connectivity with FHSS 900 MHz band', 'reason_full']

# -------------------------

In [ ]:
# Create a mapping of the (reduced) reasons captured in each enddeviceeventtypeid
ede_typeid_to_reason_df = AMIEndEvents.build_ede_typeid_to_reason_df(
    end_events_df=end_events_df, 
    reason_col='reason', 
    ede_typeid_col='enddeviceeventtypeid'
)
ede_typeid_to_reason_df['n_reasons_in_id'] = ede_typeid_to_reason_df['reason'].apply(lambda x: len(x))
ede_typeid_to_reason_df

In [ ]:
# e.g., all of the (reduced) reasons contained in enddeviceeventtypeid='3.12.136.85'
ede_typeid_to_reason_df.loc['3.12.136.85', 'reason']

In [ ]:
# Hmmm, the 'failed consecutively for 1 times for' seems a bit strange, where did it originate?
reasons_to_full_mapping.loc['failed consecutively for 1 times for', 'reason_full']

In [ ]:
end_events_df.shape

# -------------------------
# NOTE:
Below, I am only using the first 1000 entries in end_events_df.
<br>Running AMIEndEvents.get_reason_counts_per_group with the full DF takes a significant amount of time, and doesn't really add to the purpose here.

### Build the reason_counts_per_group DF grouped by trsf_pole_nb, rcpo_by_xfmr

In [ ]:
rcpo_by_xfmr = AMIEndEvents.get_reason_counts_per_group(
    end_events_df = end_events_df.iloc[:1000], 
    group_cols=['trsf_pole_nb'], 
    group_freq=None, 
    serial_number_col='serialnumber', 
    reason_col='reason', 
    include_normalize_by_nSNs=False, 
    inclue_zero_counts=True,
    possible_reasons=None, 
    include_nSNs=True, 
    include_SNs=True, 
    prem_nb_col='aep_premise_nb', 
    include_nprem_nbs=True,
    include_prem_nbs=True,   
    return_form = dict(return_multiindex_outg_reason = False, 
                       return_normalized_separately  = False)
)

In [ ]:
non_counts_cols = ['_SNs', '_nSNs', '_nprem_nbs', '_prem_nbs']
print(f'nXfmrs:                {rcpo_by_xfmr.shape[0]}')
print(f'nXfmrs w. n_events==1: {(rcpo_by_xfmr[[x for x in rcpo_by_xfmr.columns.tolist() if x not in non_counts_cols]].sum(axis=1)==1).sum()}' )
print(f'nXfmrs w. n_events>1:  {(rcpo_by_xfmr[[x for x in rcpo_by_xfmr.columns.tolist() if x not in non_counts_cols]].sum(axis=1)>1).sum()}' )
rcpo_by_xfmr

In [ ]:
end_events_df.shape

### Build the reason_counts_per_group DF grouped by serial number, rcpo_by_SN

In [ ]:
rcpo_by_SN = AMIEndEvents.get_reason_counts_per_group(
    end_events_df = end_events_df.iloc[:1000], 
    group_cols=['serialnumber'], 
    group_freq=None, 
    serial_number_col='serialnumber', 
    reason_col='reason', 
    include_normalize_by_nSNs=False, 
    inclue_zero_counts=True,
    possible_reasons=None, 
    include_nSNs=False, 
    include_SNs=False, 
    prem_nb_col='aep_premise_nb', 
    include_nprem_nbs=False,
    include_prem_nbs=False,   
    return_form = dict(return_multiindex_outg_reason = False, 
                       return_normalized_separately  = False)
)

In [ ]:
print(f'nSNs:                {rcpo_by_SN.shape[0]}')
print(f'nSNs w. n_events==1: {(rcpo_by_SN.sum(axis=1)==1).sum()}' )
print(f'nSNs w. n_events>1:  {(rcpo_by_SN.sum(axis=1)>1).sum()}' )
rcpo_by_SN

### Build the reason_counts_per_group DF grouped by serial number, and including the number of serial numbers connected to the transformer
### This is a closer mock-up (although still not exactly what we want) of what we're ultimately looking for from IT

In [ ]:
mp = MeterPremise(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    contstruct_df_args=None, 
    init_df_in_constructor=True, 
    build_sql_function=None, 
    build_sql_function_kwargs=dict(
        cols_of_interest=cols_of_interest_met_prem, 
        trsf_pole_nbs=end_events_df.iloc[:1000]['trsf_pole_nb'].unique().tolist()
    ), 
    save_args=False
)

In [ ]:
mp_df = mp.df.copy()

In [ ]:
print(mp_df.shape[0])
print(mp_df['mfr_devc_ser_nbr'].nunique())

In [ ]:
end_events_df.iloc[:1000]['mfr_devc_ser_nbr'].nunique()

In [ ]:
rcpo_final = AMIEndEvents.get_reason_counts_per_group(
    end_events_df = end_events_df.iloc[:1000], 
#     group_cols=['serialnumber'], 
    group_cols=['serialnumber', 'aep_premise_nb', 'trsf_pole_nb'], 
    group_freq=None, 
    gpby_dropna=False, 
    serial_number_col='serialnumber', 
    reason_col='reason', 
    include_normalize_by_nSNs=False, 
    inclue_zero_counts=True,
    possible_reasons=None, 
    include_nSNs=False, 
    include_SNs=False, 
    prem_nb_col='aep_premise_nb', 
    include_nprem_nbs=False,
    include_prem_nbs=False,   
    return_form = dict(return_multiindex_outg_reason = False, 
                       return_normalized_separately  = False)
)
rcpo_final=rcpo_final.reset_index().set_index('serialnumber')
#-------------------------
n_SNs_per_xfmr = mp_df.groupby('trsf_pole_nb')['mfr_devc_ser_nbr'].apply(lambda x: len(set(x)))
n_SNs_per_xfmr.name = 'n_SNs_on_xfmr'
#-------------------------
rcpo_final = pd.merge(
    rcpo_final, 
    n_SNs_per_xfmr, 
    left_on='trsf_pole_nb', 
    right_index=True, 
    how='left'
)
rcpo_final.loc[rcpo_final['n_SNs_on_xfmr'].notna(), 'n_SNs_on_xfmr'] = rcpo_final[rcpo_final['n_SNs_on_xfmr'].notna()]['n_SNs_on_xfmr'].astype(int)
rcpo_final=Utilities_df.move_cols_to_front(rcpo_final, ['aep_premise_nb', 'trsf_pole_nb', 'n_SNs_on_xfmr'])
rcpo_final['n_events_tot'] = rcpo_final.iloc[:, 3:].sum(axis=1)
rcpo_final=Utilities_df.move_cols_to_front(rcpo_final, ['aep_premise_nb', 'trsf_pole_nb', 'n_SNs_on_xfmr', 'n_events_tot'])
#-------------------------

In [ ]:
print(f"nSNs:                {rcpo_final.index.nunique()}")
print(f"nSNs w. n_events==1: {(rcpo_final['n_events_tot']==1).sum()}" )
print(f"nSNs w. n_events>1:  {(rcpo_final['n_events_tot']>1).sum()}" )
rcpo_final

In [ ]:
# rcpo_final = AMIEndEvents.get_reason_counts_per_group(
#     end_events_df = end_events_df.iloc[:1000], 
# #     group_cols=['serialnumber'], 
#     group_cols=['serialnumber', 'aep_premise_nb', 'trsf_pole_nb'], 
#     group_freq=None, 
#     gpby_dropna=False, 
#     serial_number_col='serialnumber', 
#     reason_col='reason', 
#     include_normalize_by_nSNs=False, 
#     inclue_zero_counts=True,
#     possible_reasons=None, 
#     include_nSNs=False, 
#     include_SNs=False, 
#     prem_nb_col='aep_premise_nb', 
#     include_nprem_nbs=False,
#     include_prem_nbs=False,   
#     return_form = dict(return_multiindex_outg_reason = False, 
#                        return_normalized_separately  = False)
# )
# rcpo_final=rcpo_final.reset_index().set_index('serialnumber')
# #-------------------------
# n_SNs_per_xfmr = mp_df.groupby('trsf_pole_nb')['mfr_devc_ser_nbr'].apply(lambda x: len(set(x)))
# n_SNs_per_xfmr.name = 'n_SNs_on_xfmr'
# #-------------------------
# rcpo_final = pd.merge(
#     rcpo_final, 
#     n_SNs_per_xfmr, 
#     left_on='trsf_pole_nb', 
#     right_index=True, 
#     how='left'
# )
# rcpo_final.loc[rcpo_final['n_SNs_on_xfmr'].notna(), 'n_SNs_on_xfmr'] = rcpo_final[rcpo_final['n_SNs_on_xfmr'].notna()]['n_SNs_on_xfmr'].astype(int)
# rcpo_final=Utilities_df.move_cols_to_front(rcpo_final, ['aep_premise_nb', 'trsf_pole_nb', 'n_SNs_on_xfmr'])
# rcpo_final['n_events_tot'] = rcpo_final.iloc[:, 3:].sum(axis=1)
# rcpo_final=Utilities_df.move_cols_to_front(rcpo_final, ['aep_premise_nb', 'trsf_pole_nb', 'n_SNs_on_xfmr', 'n_events_tot'])
# #-------------------------

In [ ]:
rcpo_final_xfmr = AMIEndEvents.get_reason_counts_per_group(
    end_events_df = end_events_df.iloc[:1000], 
#     group_cols=['serialnumber'], 
    group_cols=['trsf_pole_nb'], 
    group_freq=None, 
    gpby_dropna=False, 
    serial_number_col='serialnumber', 
    reason_col='reason', 
    include_normalize_by_nSNs=False, 
    inclue_zero_counts=True,
    possible_reasons=None, 
    include_nSNs=False, 
    include_SNs=False, 
    prem_nb_col='aep_premise_nb', 
    include_nprem_nbs=False,
    include_prem_nbs=False,   
    return_form = dict(return_multiindex_outg_reason = False, 
                       return_normalized_separately  = False)
)
#-------------------------
n_SNs_per_xfmr = mp_df.groupby('trsf_pole_nb')['mfr_devc_ser_nbr'].apply(lambda x: len(set(x)))
n_SNs_per_xfmr.name = 'n_SNs_on_xfmr'
#-------------------------
rcpo_final_xfmr = pd.merge(
    rcpo_final_xfmr, 
    n_SNs_per_xfmr, 
    left_on='trsf_pole_nb', 
    right_index=True, 
    how='left'
)
#-------------------------
rcpo_final_xfmr.loc[rcpo_final_xfmr['n_SNs_on_xfmr'].notna(), 'n_SNs_on_xfmr'] = rcpo_final_xfmr[rcpo_final_xfmr['n_SNs_on_xfmr'].notna()]['n_SNs_on_xfmr'].astype(int)
rcpo_final_xfmr=Utilities_df.move_cols_to_front(rcpo_final_xfmr, ['n_SNs_on_xfmr'])
rcpo_final_xfmr['n_events_tot'] = rcpo_final_xfmr.iloc[:, 1:].sum(axis=1)
rcpo_final_xfmr=Utilities_df.move_cols_to_front(rcpo_final_xfmr, ['n_SNs_on_xfmr', 'n_events_tot'])

In [ ]:
#-------------------------
rcpo_final_xfmr=rcpo_final_xfmr.loc[~rcpo_final_xfmr.index.isin(['NETWORK', 'PRIMARY', 'TRANSMISSION'])]
rcpo_final_xfmr=rcpo_final_xfmr.loc[rcpo_final_xfmr.index.notna()]

In [ ]:
rcpo_final_xfmr

In [ ]:
rcpo_final_xfmr = AMIEndEvents.get_reason_counts_per_group(
    end_events_df = end_events_df.iloc[:1000], 
#     group_cols=['serialnumber'], 
    group_cols=['trsf_pole_nb'], 
    group_freq=None, 
    gpby_dropna=False, 
    serial_number_col='serialnumber', 
    reason_col='reason', 
    include_normalize_by_nSNs=False, 
    inclue_zero_counts=True,
    possible_reasons=None, 
    include_nSNs=True, 
    include_SNs=False, 
    prem_nb_col='aep_premise_nb', 
    include_nprem_nbs=True,
    include_prem_nbs=False,   
    return_form = dict(return_multiindex_outg_reason = False, 
                       return_normalized_separately  = False)
)
#-------------------------
rcpo_final_xfmr=rcpo_final_xfmr.loc[~rcpo_final_xfmr.index.isin(['NETWORK', 'PRIMARY', 'TRANSMISSION'])]
rcpo_final_xfmr=rcpo_final_xfmr.loc[rcpo_final_xfmr.index.notna()]
#-------------------------
n_SNs_per_xfmr = mp_df.groupby('trsf_pole_nb')['mfr_devc_ser_nbr'].apply(lambda x: len(set(x)))
n_SNs_per_xfmr.name = 'n_SNs_on_xfmr'
#-------------------------
n_PNs_per_xfmr = mp_df.groupby('trsf_pole_nb')['prem_nb'].apply(lambda x: len(set(x)))
n_PNs_per_xfmr.name = 'n_PNs_on_xfmr'
#-------------------------
rcpo_final_xfmr = pd.merge(
    rcpo_final_xfmr, 
    n_SNs_per_xfmr, 
    left_on='trsf_pole_nb', 
    right_index=True, 
    how='left'
)
#-------------------------
rcpo_final_xfmr = pd.merge(
    rcpo_final_xfmr, 
    n_PNs_per_xfmr, 
    left_on='trsf_pole_nb', 
    right_index=True, 
    how='left'
)
#-------------------------
rcpo_final_xfmr.loc[rcpo_final_xfmr['n_SNs_on_xfmr'].notna(), 'n_SNs_on_xfmr'] = rcpo_final_xfmr[rcpo_final_xfmr['n_SNs_on_xfmr'].notna()]['n_SNs_on_xfmr'].astype(int)
rcpo_final_xfmr.loc[rcpo_final_xfmr['n_PNs_on_xfmr'].notna(), 'n_PNs_on_xfmr'] = rcpo_final_xfmr[rcpo_final_xfmr['n_PNs_on_xfmr'].notna()]['n_PNs_on_xfmr'].astype(int)
#-----
rcpo_final_xfmr=Utilities_df.move_cols_to_front(rcpo_final_xfmr, ['_nSNs', '_nprem_nbs', 'n_SNs_on_xfmr', 'n_PNs_on_xfmr'])
rcpo_final_xfmr['n_events_tot'] = rcpo_final_xfmr.iloc[:, 4:].sum(axis=1)
#-----
rcpo_final_xfmr=Utilities_df.move_cols_to_front(rcpo_final_xfmr, ['_nSNs', '_nprem_nbs', 'n_SNs_on_xfmr', 'n_PNs_on_xfmr', 'n_events_tot'])
#-------------------------
rcpo_final_xfmr=rcpo_final_xfmr.rename(columns={
    '_nSNs':        'n_SNs*', 
    '_nprem_nbs':   'n_PNs*', 
    'n_SNs_on_xfmr':'n_SNs', 
    'n_PNs_on_xfmr':'n_PNs'
})

In [ ]:
rcpo_final_xfmr

In [ ]:
rcpo_final_xfmr_nrm = rcpo_final_xfmr.copy()
cols_to_norm = rcpo_final_xfmr_nrm.columns[4:].tolist()
norm_col = 'n_SNs'
#-----
rcpo_final_xfmr_nrm[cols_to_norm] = rcpo_final_xfmr_nrm[cols_to_norm].divide(rcpo_final_xfmr_nrm[norm_col], axis=0)
#-----
rcpo_final_xfmr_nrm

In [ ]:
ede_1 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221014\20210101_20211231\rcpo_dfs_GRP_BY_OUTG_AND_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_df_OG.pkl')

In [ ]:
ede_1.shape

In [ ]:
ede_1

In [ ]:
ede_1.loc['3.26.38.150']['reason']

In [ ]:
# Create a mapping of the full reasons captured in each reason
reasons_to_full_mapping = end_events_df.groupby(['reason', 'enddeviceeventtypeid'])['reason_full'].unique().to_frame()
reasons_to_full_mapping['n_full_in_reason'] = reasons_to_full_mapping['reason_full'].apply(lambda x: len(x))
reasons_to_full_mapping

In [ ]:
reasons_to_full_mapping.index.get_level_values(1).nunique()

In [ ]:
reasons_to_full_mapping.index.get_level_values(1).value_counts()

In [ ]:
reasons_to_full_mapping.shape

In [ ]:
reasons_to_full_mapping

In [ ]:
df = end_events_df.copy()
reason_col='reason'
placement_col = f'{reason_col}_reduced'

patterns_to_replace=[
    r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', 
    (
        (
            r'(Under Voltage)\s*'\
            r'([0-9a-zA-Z]*)?\s*'\
            r'(\([0-9a-zA-Z\s]*\))\s*'\
            r'([0-9a-zA-Z]*)?\s?'\
            r'(for meter\:?\s*)'\
            r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
            r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
            r'(Voltage out of tolerance)?'
        ), 
        'AMIEndEvents.under_voltage_match_func'
    ), 
    (
        (
            r'(Last Gasp\s*-\s*[0-9a-zA-Z\s]*)[\s\:,.]*'\
            r'.*'\
            r'(Fail Reason: .*)$'
        ), 
        'AMIEndEvents.last_gasp_reduce_func'
    ), 
    (r'(Angle out of tolerance) \[.*\]', r'\1'), 
    (r'(NIC Power Restore Trap Received from device).*', r'\1'), #TODO This has much info, like last gasp
    (
        (
            r'(Requested operation .* could not be applied to the given device type and firmware version.) '\
            r'Device, DeviceType: .*, Firmware Version: .*$'
        ), 
        r'\1'
    ), 
    ('meterN/A', 'meter'),
    (r'(Meter needs explicit time sync.) Drift: -?\d* s, (Encountered Problems:\s*.*), Meter_Time', r'\1 \2'), 
    (r'(Meter Program Seal mismatch for Device) \[Device ID, MAC Id\] = .*', r'\1'), 
    (r'Device Time: .* Failed Device Reason: (.*) Reboot Counter: .* Refresh Counter: .*', r'\1'), 
    (r'(Ignoring (?:Interval|Register) Read data for device as it has time in the future) .*', r'\1'), 
    (r'(Secure association operation failed consecutively for 1 times for [0-9a-zA-Z]{4}.) .*', r'\1'), 
    (r'Device, (Last Gasp State: .*), (Detector State: .*), Reboot Count: \d*', r'\1, \2'), 
    (r'(Detected end of voltage sag on meter).*', r'\1'), 
    (r'(Detected end of voltage swell on meter).*', r'\1'), 
    r'N/A', 
    (r'\s{2,}', ' ')
]

In [ ]:
def reduce_end_event_reason_IT(reason, patterns, count=0, flags=re.IGNORECASE):
    r"""
    Searches for pattern in reason, if found, replaced in string.
      NOTE: Each pattern can be a single string, or a pair of strings.
      If a single string:
        the string represents the pattern and if found it is replaced by ''
      If a pair of strings:
        the first element is the pattern and the second element is the replacement
        NOTE: The replacement can be a string, a pattern, or a function!
    """
    reason_OG = reason
    matched_patterns = []
    for item in patterns:
        if Utilities.is_object_one_of_types(item, [list, tuple]):
            assert(len(item)==2)
            pattern = item[0]
            replace = item[1]
        else:
            assert(isinstance(item, str))
            pattern = item
            replace = ''
        reason_OG_i = reason
        reason = Utilities.find_and_replace_in_string(strng=reason, pattern=pattern, replace=replace, count=count, flags=flags)
        if reason_OG_i != reason:
            matched_patterns.append(pattern)
    return reason, matched_patterns

def reduce_end_event_reasons_in_df_IT(
    df, 
    reason_col='reason', 
    patterns_to_replace=[
        r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', 
        (
            (
                r'(Under Voltage)\s*'\
                r'([0-9a-zA-Z]*)?\s*'\
                r'(\([0-9a-zA-Z\s]*\))\s*'\
                r'([0-9a-zA-Z]*)?\s?'\
                r'(for meter\:?\s*)'\
                r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
                r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
                r'(Voltage out of tolerance)?'
            ), 
            'AMIEndEvents.under_voltage_match_func'
        ), 
        (
            (
                r'(Last Gasp\s*-\s*[0-9a-zA-Z\s]*)[\s\:,.]*'\
                r'.*'\
                r'(Fail Reason: .*)$'
            ), 
            'AMIEndEvents.last_gasp_reduce_func'
        ), 
        (r'(Angle out of tolerance) \[.*\]', r'\1'), 
        (r'(NIC Power Restore Trap Received from device).*', r'\1'), #TODO This has much info, like last gasp
        (
            (
                r'(Requested operation .* could not be applied to the given device type and firmware version.) '\
                r'Device, DeviceType: .*, Firmware Version: .*$'
            ), 
            r'\1'
        ), 
        ('meterN/A', 'meter'),
        (r'(Meter needs explicit time sync.) Drift: -?\d* s, (Encountered Problems:\s*.*), Meter_Time', r'\1 \2'), 
        (r'(Meter Program Seal mismatch for Device) \[Device ID, MAC Id\] = .*', r'\1'), 
        (r'Device Time: .* Failed Device Reason: (.*) Reboot Counter: .* Refresh Counter: .*', r'\1'), 
        (r'(Ignoring (?:Interval|Register) Read data for device as it has time in the future) .*', r'\1'), 
        (r'(Secure association operation failed consecutively for 1 times for [0-9a-zA-Z]{4}.) .*', r'\1'), 
        (r'Device, (Last Gasp State: .*), (Detector State: .*), Reboot Count: \d*', r'\1, \2'), 
        (r'(Detected end of voltage sag on meter).*', r'\1'), 
        (r'(Detected end of voltage swell on meter).*', r'\1'), 
        r'N/A', 
        (r'\s{2,}', ' ')
    ], 
    addtnl_patterns_to_replace=None, 
    count=0, 
    flags=re.IGNORECASE, 
    remove_trailing_punctuation=True, 
    inplace=True
):
    r"""
    Searches for each of patterns (= patterns_to_replace + addtnl_patterns_to_replace) in reason, if found, replaced in string.
      NOTE: Each pattern can be a single string, or a pair of strings.
      If a single string:
        the string represents the pattern and if found it is replaced by ''
      If a pair of strings:
        the first element is the pattern and the second element is the replacement
        NOTE: The replacement can be a string, a pattern, or a function!
        NOTE: Needed to be a little creative with the treatment of AMIEndEvents.under_voltage_match_func and AMIEndEvents.last_gasp_reduce_func.
              Forward declaration is not possible in Python, and putting these directly into the default patterns_to_replace argument causes an error.
              Therefore, I use the strings 'AMIEndEvents.under_voltage_match_func' and 'AMIEndEvents.last_gasp_reduce_func' in the default argument, 
              and substitute within the body of the function.
              If one feeds in own patterns_to_replace in some outside call (e.g., Jupyter notebook, some other py file, etc.), using 
              AMIEndEvents.under_voltage_match_func and AMIEndEvents.last_gasp_reduce_func without quotes should work fine.


    e.g., with default patterns:
        i.  
          'Under Voltage (CA000400) cleared for meter 00:13:50:01:00:a7:bc:52.'
    ==>   'Under Voltage (CA000400) cleared'
        ii.  
          'Last Gasp - NIC power lost for device: 00:13:50:02:00:9c:8f:2b, Reboot Count: 194, NIC timestamp: Jan 1, 2018 - 
           02:14:19 AM EST, Received timestamp: Jan 1, 2018 - 02:14:27 AM EST, Fail Reason: NIC power fail'
    ==>   'Last Gasp - NIC power lost'
        iii.  
          'Under Voltage (CA000400) cleared for meter 00:13:50:01:00:a7:bc:52. for device 00:13:50:01:00:a7:bc:52.'
    ==>   'Under Voltage (CA000400) cleared'

    ---------------------------------------------
    Description/motivation for default patterns:
    ---------------------------------------------
    - r'\:?\s?([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+'
      Matches all MAC-esque meter/device IDs
      NOTE: + used at end because want ONE OR MORE repetitions of (\:[0-9a-zA-Z]{1,2})
            * matches ZERO OR MORE 
            EXAMPLE: Assume strng = 'Demand Reset occurred for meter 00:13:50:01:00:17:c6:41.'
              If * used: match = 'De'
              If + used: match = ' 00:13:50:01:00:17:c6:41'
      EX:
            'Under Voltage (CA000400) cleared for meter 00:13:50:01:00:a7:bc:52.'
        ==> 'Under Voltage (CA000400) cleared for meter.'
    """
    #-------------------------
    if not inplace:
        df = df.copy()
    #-------------------------
    placement_col = f'{reason_col}_reduced'
    if placement_col in df.columns.tolist():
        placement_col += '_'+Utilities.generate_random_string()
    assert(placement_col not in df.columns.tolist())
    #-------------------------
    if addtnl_patterns_to_replace is not None:
        patterns_to_replace.extend(addtnl_patterns_to_replace)
    #-------------------------
    # Creative way of smuggling in AMIEndEvents.under_voltage_match_func and AMIEndEvents.last_gasp_reduce_func in 
    # the patterns_to_replace default parameter.  See NOTE above.
    for i in range(len(patterns_to_replace)):
        if patterns_to_replace[i][1]=='AMIEndEvents.under_voltage_match_func':
            patterns_to_replace[i] = (patterns_to_replace[i][0], AMIEndEvents.under_voltage_match_func)
        if patterns_to_replace[i][1]=='AMIEndEvents.last_gasp_reduce_func':
            patterns_to_replace[i] = (patterns_to_replace[i][0], AMIEndEvents.last_gasp_reduce_func)        
    #-------------------------
    # This would be handled in reduce_end_event_reason if not here, but doing it once here
    # should save time, especially for large DFs, I believe.
    for i in range(len(patterns_to_replace)):
        pattern = patterns_to_replace[i]
        if Utilities.is_object_one_of_types(pattern, [list, tuple]):
            assert(len(pattern)==2)
            continue
        else:
            assert(isinstance(pattern, str))
            patterns_to_replace[i] = (pattern, '')
    #-------------------------
    df_addtnl = df.apply(lambda x: reduce_end_event_reason_IT(x['reason'], patterns_to_replace), axis=1, result_type='expand')
    assert(df_addtnl.shape[1]==2)
    assert(df_addtnl.columns.tolist()==[0,1])
    #-----
    matched_patterns_col = 'matched_patterns'
    if matched_patterns_col in df.columns.tolist():
        matched_patterns_col += '_'+Utilities.generate_random_string()
    assert(matched_patterns_col not in df.columns.tolist())
    df_addtnl=df_addtnl.rename(columns={0:placement_col, 1:matched_patterns_col})        
    #-----
    n_matched_col = 'n_matched'
    if n_matched_col in df.columns.tolist():
        n_matched_col += '_'+Utilities.generate_random_string()
    assert(n_matched_col not in df.columns.tolist())
    df_addtnl[n_matched_col] = df_addtnl[matched_patterns_col].apply(lambda x: len(x))        
    #-----
    assert(df.shape[0]==df_addtnl.shape[0])
    assert(all(df.index==df_addtnl.index))
    df = pd.merge(df, df_addtnl, left_index=True, right_index=True, how='inner')        
    #-------------------------
    if remove_trailing_punctuation:
        df[placement_col] = df[placement_col].apply(lambda x: AMIEndEvents.remove_trailing_punctuation_from_reason(x))
    return df

In [ ]:
end_events_df = reduce_end_event_reasons_in_df_IT(
    df=end_events_df, 
    patterns_to_replace=patterns_to_replace, 
    reason_col='reason'
)

In [ ]:
print(f"# Events with 0 regex matches:  {(end_events_df['n_matched']==0).sum()}")
print(f"# Events with 1 regex matches:  {(end_events_df['n_matched']==1).sum()}")
print(f"# Events with 2 regex matches:  {(end_events_df['n_matched']==2).sum()}")
print(f"# Events with >2 regex matches: {(end_events_df['n_matched']>2).sum()}")

In [ ]:
any(end_events_df['n_matched']>1)

In [ ]:
end_events_df[end_events_df['n_matched']>1]

In [ ]:
end_events_df[end_events_df['n_matched']>1].iloc[0]['matched_patterns']

In [ ]:
gt1_match = end_events_df[end_events_df['n_matched']>1].copy()

In [ ]:
gt1_match['matched_patterns'].explode().shape

In [ ]:
gt1_match.shape

In [ ]:
2*gt1_match.shape[0]

In [ ]:
list(set(gt1_match['matched_patterns'].explode().tolist()))

In [ ]:
dev_df = end_events_df[['enddeviceeventtypeid', 'reason', 'reason_reduced', 'matched_patterns', 'n_matched']].copy()
dev_df

In [ ]:
# If a reason matches with only one regex, it is always the MAC-esque address reduction
dev_df[dev_df['n_matched']==1].explode('matched_patterns')['matched_patterns'].unique()

In [ ]:
dev_df.explode('matched_patterns').iloc[4]

In [ ]:
dev_df.explode('matched_patterns').iloc[5]

In [ ]:
dev_df.iloc[4]['matched_patterns']

In [ ]:
hmm=dev_df.explode('matched_patterns').groupby('enddeviceeventtypeid')['matched_patterns'].unique()
hmm=hmm.sort_index()
hmm

In [ ]:
type(hmm)

In [ ]:
hmm.iloc[0]

In [ ]:
for idx, patterns in hmm.items():
    print(idx)
    print(f'# regex matches = {len(patterns)}')
    for pattern in patterns:
        print(pattern)
    print('\n\n')

In [ ]:
# # ede_df = GenAn.read_df_from_csv_dir_batches(
# #     files_dir=r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221014\20210101_20211231\EndEvents_NoOutg', 
# #     file_path_glob=r'end_events_[0-9]*.csv', 
# #     file_path_regex=None, 
# # )

# # ede_df = GenAn.read_df_from_csv_dir_batches(
# #     files_dir=r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20210101_20211231\EndEvents_NoOutg', 
# #     file_path_glob=r'end_events_[0-9]*.csv', 
# #     file_path_regex=None, 
# # )

# ede_df = GenAn.read_df_from_csv_dir_batches(
#     files_dir=r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20220101_20220930\EndEvents_NoOutg', 
#     file_path_glob=r'end_events_[0-9]*.csv', 
#     file_path_regex=None, 
# )



ede_df_1 = GenAn.read_df_from_csv_dir_batches(
    files_dir=r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221014\20210101_20211231\EndEvents_NoOutg', 
    file_path_glob=r'end_events_[0-9]*.csv', 
    file_path_regex=None, 
)
ede_df_2 = GenAn.read_df_from_csv_dir_batches(
    files_dir=r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20210101_20211231\EndEvents_NoOutg', 
    file_path_glob=r'end_events_[0-9]*.csv', 
    file_path_regex=None, 
)
ede_df_3 = GenAn.read_df_from_csv_dir_batches(
    files_dir=r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20220101_20220930\EndEvents_NoOutg', 
    file_path_glob=r'end_events_[0-9]*.csv', 
    file_path_regex=None, 
)
#-----
ede_df_cols = set.intersection(set(ede_df_1.columns.tolist()), set(ede_df_2.columns.tolist()), set(ede_df_3.columns.tolist()))
ede_df_OG = pd.concat([ede_df_1[ede_df_cols], ede_df_2[ede_df_cols], ede_df_3[ede_df_cols]])

In [ ]:
ede_df = ede_df_OG.copy()

In [ ]:
# BELOW USED TOO MUCH MEMORY APPARENTLY!!!!!!!!!!!!!!!!!!!!
# ede_df = reduce_end_event_reasons_in_df_IT(
#     df=ede_df, 
#     patterns_to_replace=patterns_to_replace, 
#     reason_col='reason_full'
# )

In [ ]:
patterns_to_replace=[
    r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', 
    (
        (
            r'(Under Voltage)\s*'\
            r'([0-9a-zA-Z]*)?\s*'\
            r'(\([0-9a-zA-Z\s]*\))\s*'\
            r'([0-9a-zA-Z]*)?\s?'\
            r'(for meter\:?\s*)'\
            r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
            r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
            r'(Voltage out of tolerance)?'
        ), 
        'AMIEndEvents.under_voltage_match_func'
    ), 
    (
        (
            r'(Last Gasp\s*-\s*[0-9a-zA-Z\s]*)[\s\:,.]*'\
            r'.*'\
            r'(Fail Reason: .*)$'
        ), 
        'AMIEndEvents.last_gasp_reduce_func'
    ), 
    (r'(Angle out of tolerance) \[.*\]', r'\1'), 
    (r'(NIC Power Restore Trap Received from device).*', r'\1'), #TODO This has much info, like last gasp
    (
        (
            r'(Requested operation .* could not be applied to the given device type and firmware version.) '\
            r'Device, DeviceType: .*, Firmware Version: .*$'
        ), 
        r'\1'
    ), 
    ('meterN/A', 'meter'),
    (r'(Meter needs explicit time sync.) Drift: -?\d* s, (Encountered Problems:\s*.*), Meter_Time', r'\1 \2'), 
    (r'(Meter Program Seal mismatch for Device) \[Device ID, MAC Id\] = .*', r'\1'), 
    (r'Device Time: .* Failed Device Reason: (.*) Reboot Counter: .* Refresh Counter: .*', r'\1'), 
    (r'(Ignoring (?:Interval|Register) Read data for device as it has time in the future) .*', r'\1'), 
    (r'(Secure association operation failed consecutively for 1 times for [0-9a-zA-Z]{4}.) .*', r'\1'), 
    (r'Device, (Last Gasp State: .*), (Detector State: .*), Reboot Count: \d*', r'\1, \2'), 
    (r'(Detected end of voltage sag on meter).*', r'\1'), 
    (r'(Detected end of voltage swell on meter).*', r'\1'), 
    r'N/A', 
    (r'\s{2,}', ' ')
]

In [ ]:
ressys = []
for idx, row in ede_df.iterrows():
    typeid      = row['enddeviceeventtypeid']
    reason_full = row['reason']
    reason, matched_patterns = reduce_end_event_reason_IT(reason_full, patterns_to_replace)
    ressys.append([typeid, reason_full, reason, matched_patterns])

In [ ]:
len(ressys)

In [ ]:
ressys

In [ ]:
ressys_df = pd.DataFrame(ressys)
ressys_df

In [ ]:
ressys_df = ressys_df.rename(columns={
    0:'enddeviceeventtypeid', 
    1:'reason', 
    2:'reason_reduced', 
    3:'matched_patterns'
})

In [ ]:
ressys_df['n_matched'] = ressys_df['matched_patterns'].apply(lambda x: len(x))

In [ ]:
ressys_df

In [ ]:
print(f"# Events with 0 regex matches:  {(ressys_df['n_matched']==0).sum()}")
print(f"# Events with 1 regex matches:  {(ressys_df['n_matched']==1).sum()}")
print(f"# Events with 2 regex matches:  {(ressys_df['n_matched']==2).sum()}")
print(f"# Events with >2 regex matches: {(ressys_df['n_matched']>2).sum()}")

In [ ]:
# If a reason matches with only one regex, it is always the MAC-esque address reduction
ressys_df[ressys_df['n_matched']==1].explode('matched_patterns')['matched_patterns'].unique()

In [ ]:
hmm2=ressys_df.explode('matched_patterns').groupby('enddeviceeventtypeid')['matched_patterns'].unique()
hmm2=hmm2.sort_index()
hmm2

In [ ]:
hmm2.shape

In [ ]:
hmm.shape

In [ ]:
for idx, patterns in hmm2.items():
    print(idx)
    print(f'# regex matches = {len(patterns)}')
    for pattern in patterns:
        print(pattern)
    print('\n\n')

In [ ]:
# # ede_typeid_to_reason_df = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221014\20210101_20211231\rcpo_dfs_GRP_BY_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_no_outg_df_OG.pkl')
# # ede_typeid_to_reason_df = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20210101_20211231\rcpo_dfs_GRP_BY_NO_OUTG_AND_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_no_outg_df_OG.pkl')
# ede_typeid_to_reason_df = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20220101_20220930\rcpo_dfs_GRP_BY_NO_OUTG_AND_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_no_outg_df_OG.pkl')
# ede_typeid_to_reason_df['n_reasons_in_id'] = ede_typeid_to_reason_df['reason'].apply(lambda x: len(x))

ede_typeid_to_reason_df_1 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221014\20210101_20211231\rcpo_dfs_GRP_BY_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_no_outg_df_OG.pkl')
ede_typeid_to_reason_df_2 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20210101_20211231\rcpo_dfs_GRP_BY_NO_OUTG_AND_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_no_outg_df_OG.pkl')
ede_typeid_to_reason_df_3 = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20220101_20220930\rcpo_dfs_GRP_BY_NO_OUTG_AND_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_no_outg_df_OG.pkl')
#-----
ede_typeid_to_reason_df = AMIEndEvents.combine_two_ede_typeid_to_reason_dfs(
    ede_typeid_to_reason_df1=ede_typeid_to_reason_df_1, 
    ede_typeid_to_reason_df2=ede_typeid_to_reason_df_2,
    sort=True
)

ede_typeid_to_reason_df = AMIEndEvents.combine_two_ede_typeid_to_reason_dfs(
    ede_typeid_to_reason_df1=ede_typeid_to_reason_df, 
    ede_typeid_to_reason_df2=ede_typeid_to_reason_df_3,
    sort=True
)
#-----
ede_typeid_to_reason_df['n_reasons_in_id'] = ede_typeid_to_reason_df['reason'].apply(lambda x: len(x))

In [ ]:
ede_typeid_to_reason_df

In [ ]:
ede_typeid_to_reason_df.shape

In [ ]:
for idx, patterns in hmm2.items():
    print('*****'*5)
    print(idx)
    print('*****'*5)
    print(f'# regex matches = {len(patterns)}')
    for pattern in patterns:
        print(pattern)
    print('-----'*5)
    reasons_red = ede_typeid_to_reason_df.loc[idx]['reason']
    print(f'# reduced reasons = {len(reasons_red)}')
    for reason_red in reasons_red:
        print(reason_red)
    print('\n\n')

In [ ]:
ede_typeid_to_reason_df.loc['3.26.136.66']['reason']

In [ ]:
for idx, patterns in hmm2.items():
    if len(patterns)==1:
        continue
    print('*****'*5)
    print(idx)
    print('*****'*5)
    print(f'# regex matches = {len(patterns)}')
    for pattern in patterns:
        print(pattern)
    print('-----'*5)
    reasons_red = ede_typeid_to_reason_df.loc[idx]['reason']
    print(f'# reduced reasons = {len(reasons_red)}')
    for reason_red in reasons_red:
        print(reason_red)
    print('\n\n')

In [ ]:
for idx, patterns in hmm2.items():
    if len(patterns)!=1:
        continue
    print('*****'*5)
    print(idx)
    print('*****'*5)
    print(f'# regex matches = {len(patterns)}')
    for pattern in patterns:
        print(pattern)
    print('-----'*5)
    reasons_red = ede_typeid_to_reason_df.loc[idx]['reason']
    print(f'# reduced reasons = {len(reasons_red)}')
    for reason_red in reasons_red:
        print(reason_red)
    print('\n\n')

In [ ]:
def get_std_patterns_to_replace_by_typeid_EXPLICIT_OG():
    r"""
    Returns the standard patterns_to_replace_by_typeid.
    Function exists essentially to keep reduce_end_event_reasons_in_df_v2 from being too cluttered
    
    NOTE: Multi-line strings have more parentheses than absolutely necessary for clarity.
    NOTE: In (r'\s{2,}', ' '), the second element ' ' has a space between the quotes! So, multiple white-space characters
            are to be replaced with a single-space.
          This is different from many other cases, where ther is no space between the quotes, and in which case the match
            is to be removed (by replacing it with an empty character, e.g., (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''))  
    """
    #-------------------------
    patterns_to_replace_by_typeid = {
#         # The following will be run at the beginning for all
#         'all_beg':          [
#             (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', '')
#         ], 
        
#         # The following will be run at the end for all
#         # 0th element:
#         #   The pattern removes any of the characters between [] (i.e., !"#$%&'(*+,-./:;<=>?@[\^_`{|~), 
#         #     as many times as they show up, but ONLY at the end of the string
#         # 1st element:
#         #   Replace multiple spaces with single space
#         'all_end':          [
#             (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
#             (r'\s{2,}', ' ')
#         ], 

        # total regex = 5
        '3.11.63.161':  [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (
                (
                    r'(Requested operation .* could not be applied to the given device type and firmware version.) '\
                    r'Device, DeviceType: .*, Firmware Version: .*$'
                ), 
                r'\1'
            ), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''),
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.12.136.38':  [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'N/A', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 6
        '3.12.136.85':  [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'(Secure association operation failed consecutively for 1 times for [0-9a-zA-Z]{4}.) .*', r'\1'), 
            #-----
            (r'N/A', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.12.17.257':  [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.12.48.219':  [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'(Angle out of tolerance) \[.*\]', r'\1'), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.12.48.28':   [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.12.93.219':  [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.12.93.28':   [
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.18.72.28':   [
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.18.72.79':   [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.21.3.79':    [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.21.38.223':  [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'(Detected end of voltage sag on meter).*', r'\1'), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.21.38.248':  [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'(Detected end of voltage swell on meter).*', r'\1'), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.23.136.47':  [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.23.136.85':  [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'Device Time: .* Failed Device Reason: (.*) Reboot Counter: .* Refresh Counter: .*', r'\1'), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.23.17.139':  [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.26.0.216':   [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.26.0.47':    [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.26.136.216': [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'(NIC Power Restore Trap Received from device).*', r'\1'), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.26.136.47':  [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (
                (
                    r'(Last Gasp\s*-\s*[0-9a-zA-Z\s]*)[\s\:,.]*'\
                    r'.*'\
                    r'(Fail Reason: .*)$'
                ), 
                r'\1, \2'
            ), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.26.136.66':  [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'Device, (Last Gasp State: .*), (Detector State: .*), Reboot Count: \d*', r'\1, \2'), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 6
        '3.26.38.150':  [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (
                (
                    r'(Under Voltage)\s*'      
                    r'(\([0-9a-zA-Z\s]*\))\s*'\
                    r'([0-9a-zA-Z]*)?\s?'\
                    r'(for meter\:?\s*)'\
                    r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
                    r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
                    r'(Voltage out of tolerance)?'
                ), 
                r'\1 \2 \3 \4 \5'
            ), 
            #-----
            (
                (
                    r'(Under Voltage)\s*'
                    r'([0-9a-zA-Z]*)?\s*'\
                    r'(\([0-9a-zA-Z\s]*\))\s*'\
                    r'(for meter\:?\s*)'\
                    r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
                    r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
                    r'(Voltage out of tolerance)?'
                ), 
                r'\1 \3 \2 \4 \5'
            ), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 8
        '3.26.38.37':   [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (
                (
                    r'(Under Voltage)\s*'      
                    r'(\([0-9a-zA-Z\s]*\))\s*'\
                    r'([0-9a-zA-Z]*)?\s?'\
                    r'(for meter\:?\s*)'\
                    r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
                    r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
                    r'(Voltage out of tolerance)?'
                ), 
                r'\1 \2 \3 \4 \5'
            ), 
            #-----
            (
                (
                    r'(Under Voltage)\s*'
                    r'([0-9a-zA-Z]*)?\s*'\
                    r'(\([0-9a-zA-Z\s]*\))\s*'\
                    r'(for meter\:?\s*)'\
                    r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
                    r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
                    r'(Voltage out of tolerance)?'
                ), 
                r'\1 \3 \2 \4 \5'
            ), 
            #-----
            ('meterN/A', 'meter'), 
            #-----
            (r'N/A', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.26.38.47':   [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.26.38.73':   [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            ('meterN/A', 'meter'), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.26.38.93':   [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.35.0.28':    [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.35.0.40':    [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.36.114.159': [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'(Ignoring (?:Interval|Register) Read data for device as it has time in the future) .*', r'\1'), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.36.114.73':  [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.36.136.73':  [
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'(Meter needs explicit time sync.) Drift: -?\d* s, (Encountered Problems:\s*.*), Meter_Time', r'\1 \2'), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.36.136.79':  [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.7.19.242':   [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.7.19.243':   [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.8.0.215':    [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.9.83.159':   [
            #-----
            (r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', ''), 
            #-----
            (r'(Meter Program Seal mismatch for Device) \[Device ID, MAC Id\] = .*', r'\1'), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ]
    }
    #-------------------------
    return patterns_to_replace_by_typeid

In [ ]:
def get_std_patterns_to_replace_by_typeid_OG():
    r"""
    Returns the standard patterns_to_replace_by_typeid.
    Function exists essentially to keep reduce_end_event_reasons_in_df_v2 from being too cluttered
    
    NOTE: Multi-line strings have more parentheses than absolutely necessary for clarity
    """
    #-------------------------
    patterns_to_replace_by_typeid = {
        # The following will be run at the beginning for all
        # 0th element: for removing MAC-esque meter codes
        'all_beg':          [
            #-----
            r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+'
            #-----
        ], 
        
        # The following will be run at the end for all
        # 0th element:
        #   The pattern removes any of the characters between [] (i.e., !"#$%&'(*+,-./:;<=>?@[\^_`{|~), 
        #     as many times as they show up, but ONLY at the end of the string
        # 1st element:
        #   Replace multiple spaces with single space
        'all_end':          [
            #-----
            r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ], 

        # addtnl regex = 1
        # total regex = 5
        '3.11.63.161':  [
            #-----
            (
                (
                    r'(Requested operation .* could not be applied to the given device type and firmware version.) '\
                    r'Device, DeviceType: .*, Firmware Version: .*$'
                ), 
                r'\1'
            )
            #-----
        ],

        # addtnl regex = 1
        # total regex = 5
        '3.12.136.38':  [
            #-----
            r'N/A'
            #-----
        ],

        # addtnl regex = 2
        # total regex = 6
        '3.12.136.85':  [
            #-----
            (r'(Secure association operation failed consecutively for 1 times for [0-9a-zA-Z]{4}.) .*', r'\1'), 
            #-----
            r'N/A'
            #-----
        ],

        # addtnl regex = 0
        # total regex = 4
        '3.12.17.257':  [],

        # addtnl regex = 1
        # total regex = 5
        '3.12.48.219':  [
            #-----
            (r'(Angle out of tolerance) \[.*\]', r'\1')
            #-----
        ],

        # addtnl regex = 0
        # total regex = 4
        '3.12.48.28':   [],

        # addtnl regex = 0
        # total regex = 4
        '3.12.93.219':  [],

        # addtnl regex = 0
        # total regex = 4
        '3.12.93.28':   [],

        # addtnl regex = 0
        # total regex = 4
        '3.18.72.28':   [],

        # addtnl regex = 0
        # total regex = 4
        '3.18.72.79':   [],

        # addtnl regex = 0
        # total regex = 4
        '3.21.3.79':    [],

        # addtnl regex = 1
        # total regex = 5
        '3.21.38.223':  [
            #-----
            (r'(Detected end of voltage sag on meter).*', r'\1')
            #-----
        ],

        # addtnl regex = 1
        # total regex = 5
        '3.21.38.248':  [
            #-----
            (r'(Detected end of voltage swell on meter).*', r'\1')
            #-----
        ],

        # addtnl regex = 0
        # total regex = 4
        '3.23.136.47':  [],

        # addtnl regex = 1
        # total regex = 5
        '3.23.136.85':  [
            #-----
            (r'Device Time: .* Failed Device Reason: (.*) Reboot Counter: .* Refresh Counter: .*', r'\1')
            #-----
        ],

        # addtnl regex = 0
        # total regex = 4
        '3.23.17.139':  [],

        # addtnl regex = 0
        # total regex = 4
        '3.26.0.216':   [],

        # addtnl regex = 0
        # total regex = 4
        '3.26.0.47':    [],

        # addtnl regex = 1
        # total regex = 5
        '3.26.136.216': [
            #-----
            (r'(NIC Power Restore Trap Received from device).*', r'\1')
            #-----
        ],

        # addtnl regex = 1
        # total regex = 5
        '3.26.136.47':  [
            #-----
            (
                (
                    r'(Last Gasp\s*-\s*[0-9a-zA-Z\s]*)[\s\:,.]*'\
                    r'.*'\
                    r'(Fail Reason: .*)$'
                ), 
                r'\1, \2'
            )
            #-----
        ],

        # addtnl regex = 1
        # total regex = 5
        '3.26.136.66':  [
            #-----
            (r'Device, (Last Gasp State: .*), (Detector State: .*), Reboot Count: \d*', r'\1, \2')
            #-----
        ],

        # addtnl regex = 2
        # total regex = 6
        '3.26.38.150':  [
            #-----
            (
                (
                    r'(Under Voltage)\s*'      
                    r'(\([0-9a-zA-Z\s]*\))\s*'\
                    r'([0-9a-zA-Z]*)?\s?'\
                    r'(for meter\:?\s*)'\
                    r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
                    r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
                    r'(Voltage out of tolerance)?'
                ), 
                r'\1 \2 \3 \4 \5'
            ), 
            #-----
            (
                (
                    r'(Under Voltage)\s*'
                    r'([0-9a-zA-Z]*)?\s*'\
                    r'(\([0-9a-zA-Z\s]*\))\s*'\
                    r'(for meter\:?\s*)'\
                    r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
                    r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
                    r'(Voltage out of tolerance)?'
                ), 
                r'\1 \3 \2 \4 \5'
            )
            #-----
        ],

        # addtnl regex = 4
        # total regex = 8
        '3.26.38.37':   [
            #-----
            (
                (
                    r'(Under Voltage)\s*'      
                    r'(\([0-9a-zA-Z\s]*\))\s*'\
                    r'([0-9a-zA-Z]*)?\s?'\
                    r'(for meter\:?\s*)'\
                    r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
                    r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
                    r'(Voltage out of tolerance)?'
                ), 
                r'\1 \2 \3 \4 \5'
            ), 
            #-----
            (
                (
                    r'(Under Voltage)\s*'
                    r'([0-9a-zA-Z]*)?\s*'\
                    r'(\([0-9a-zA-Z\s]*\))\s*'\
                    r'(for meter\:?\s*)'\
                    r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
                    r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
                    r'(Voltage out of tolerance)?'
                ), 
                r'\1 \3 \2 \4 \5'
            ), 
            #-----
            ('meterN/A', 'meter'), 
            #-----
            r'N/A'
            #-----
        ],

        # addtnl regex = 0
        # total regex = 4
        '3.26.38.47':   [],

        # addtnl regex = 1
        # total regex = 5
        '3.26.38.73':   [
            #-----
            ('meterN/A', 'meter')
            #-----
        ],

        # addtnl regex = 0
        # total regex = 4
        '3.26.38.93':   [],

        # addtnl regex = 0
        # total regex = 4
        '3.35.0.28':    [],

        # addtnl regex = 0
        # total regex = 4
        '3.35.0.40':    [],

        # addtnl regex = 1
        # total regex = 5
        '3.36.114.159': [
            #-----
            (r'(Ignoring (?:Interval|Register) Read data for device as it has time in the future) .*', r'\1')
            #-----
        ],

        # addtnl regex = 0
        # total regex = 4
        '3.36.114.73':  [],

        # addtnl regex = 1
        # total regex = 5
        '3.36.136.73':  [
            #-----
            (r'(Meter needs explicit time sync.) Drift: -?\d* s, (Encountered Problems:\s*.*), Meter_Time', r'\1 \2')
            #-----
        ],

        # addtnl regex = 0
        # total regex = 4
        '3.36.136.79':  [],

        # addtnl regex = 0
        # total regex = 4
        '3.7.19.242':   [],

        # addtnl regex = 0
        # total regex = 4
        '3.7.19.243':   [],

        # addtnl regex = 0
        # total regex = 4
        '3.8.0.215':    [],

        # addtnl regex = 1
        # total regex = 5
        '3.9.83.159':   [
            #-----
            (r'(Meter Program Seal mismatch for Device) \[Device ID, MAC Id\] = .*', r'\1')
            #-----
        ]
    }
    #-------------------------
    return patterns_to_replace_by_typeid

In [ ]:
def get_std_patterns_to_replace_by_typeid_EXPLICIT():
    r"""
    Returns the standard patterns_to_replace_by_typeid.
    Function exists essentially to keep reduce_end_event_reasons_in_df_v2 from being too cluttered
    
    NOTE: Multi-line strings have more parentheses than absolutely necessary for clarity.
    NOTE: In (r'\s{2,}', ' '), the second element ' ' has a space between the quotes! So, multiple white-space characters
            are to be replaced with a single-space.
          This is different from many other cases, where ther is no space between the quotes, and in which case the match
            is to be removed (by replacing it with an empty character, e.g., (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''))  
    """
    #-------------------------
    patterns_to_replace_by_typeid = {
#         # The following will be run at the beginning for all
#         'all_beg':          [
#             (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', '')
#         ], 
        
#         # The following will be run at the end for all
#         # 0th element:
#         #   The pattern removes any of the characters between [] (i.e., !"#$%&'(*+,-./:;<=>?@[\^_`{|~), 
#         #     as many times as they show up, but ONLY at the end of the string
#         # 1st element:
#         #   Replace multiple spaces with single space
#         'all_end':          [
#             (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
#             (r'\s{2,}', ' ')
#         ], 

        # total regex = 5
        '3.11.63.161':  [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (
                (
                    r'(Requested operation .* could not be applied to the given device type and firmware version.) '\
                    r'Device, DeviceType: .*, Firmware Version: .*$'
                ), 
                r'\1'
            ), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''),
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.12.136.38':  [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'N/A', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 6
        '3.12.136.85':  [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'(Secure association operation failed consecutively for 1 times for [0-9a-zA-Z]{4}.) .*', r'\1'), 
            #-----
            (r'N/A', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.12.17.257':  [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.12.48.219':  [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'(Angle out of tolerance) \[.*\]', r'\1'), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.12.48.28':   [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.12.93.219':  [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.12.93.28':   [
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.18.72.28':   [
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.18.72.79':   [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.21.3.79':    [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.21.38.223':  [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'(Detected end of voltage sag on meter).*', r'\1'), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.21.38.248':  [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'(Detected end of voltage swell on meter).*', r'\1'), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.23.136.47':  [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.23.136.85':  [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'Device Time: .* Failed Device Reason: (.*) Reboot Counter: .* Refresh Counter: .*', r'\1'), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.23.17.139':  [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.26.0.216':   [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.26.0.47':    [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.26.136.216': [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'(NIC Power Restore Trap Received from device).*', r'\1'), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.26.136.47':  [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (
                (
                    r'(Last Gasp\s*-\s*[0-9a-zA-Z\s]*)[\s\:,.]*'\
                    r'.*'\
                    r'(Fail Reason: .*)$'
                ), 
                r'\1, \2'
            ), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.26.136.66':  [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'Device, (Last Gasp State: .*), (Detector State: .*), Reboot Count: \d*', r'\1, \2'), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 6
        '3.26.38.150':  [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (
                (
                    r'(Under Voltage)\s*'      
                    r'(\([0-9a-zA-Z\s]*\))\s*'\
                    r'([0-9a-zA-Z]*)?\s?'\
                    r'(for meter\:?\s*)'\
                    r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
                    r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
                    r'(Voltage out of tolerance)?'
                ), 
                r'\1 \2 \3 \4 \5'
            ), 
            #-----
            (
                (
                    r'(Under Voltage)\s*'
                    r'([0-9a-zA-Z]*)?\s*'\
                    r'(\([0-9a-zA-Z\s]*\))\s*'\
                    r'(for meter\:?\s*)'\
                    r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
                    r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
                    r'(Voltage out of tolerance)?'
                ), 
                r'\1 \3 \2 \4 \5'
            ), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 8
        '3.26.38.37':   [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (
                (
                    r'(Under Voltage)\s*'      
                    r'(\([0-9a-zA-Z\s]*\))\s*'\
                    r'([0-9a-zA-Z]*)?\s?'\
                    r'(for meter\:?\s*)'\
                    r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
                    r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
                    r'(Voltage out of tolerance)?'
                ), 
                r'\1 \2 \3 \4 \5'
            ), 
            #-----
            (
                (
                    r'(Under Voltage)\s*'
                    r'([0-9a-zA-Z]*)?\s*'\
                    r'(\([0-9a-zA-Z\s]*\))\s*'\
                    r'(for meter\:?\s*)'\
                    r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
                    r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
                    r'(Voltage out of tolerance)?'
                ), 
                r'\1 \3 \2 \4 \5'
            ), 
            #-----
            ('meterN/A', 'meter'), 
            #-----
            (r'N/A', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.26.38.47':   [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.26.38.73':   [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            ('meterN/A', 'meter'), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.26.38.93':   [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.35.0.28':    [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.35.0.40':    [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.36.114.159': [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'(Ignoring (?:Interval|Register) Read data for device as it has time in the future) .*', r'\1'), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.36.114.73':  [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.36.136.73':  [
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'(Meter needs explicit time sync.) Drift: -?\d* s, (Encountered Problems:\s*.*), Meter_Time', r'\1 \2'), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.36.136.79':  [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.7.19.242':   [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.7.19.243':   [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 4
        '3.8.0.215':    [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],

        # total regex = 5
        '3.9.83.159':   [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'(Meter Program Seal mismatch for Device) \[Device ID, MAC Id\] = .*', r'\1'), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ], 
        
        # total regex = 7
        '3.18.72.85':   [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'(Meter event Nonvolatile Memory Failure Detected)\s*Time event occurred on meter = .* Sequence number = .* User id = .*  Event argument = .*', r'\1'), 
            #-----
            (r'(Meter event Nonvolatile Memory Failure Detected).*', r'\1'),
            #-----
            (r'(Meter detected a nonvolatile memory failure).*', 'Meter event Nonvolatile Memory Failure Detected'), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ], 
        
        # total regex = 4
        '3.21.17.28':    [
            #-----
            (r'\s+([0-9a-zA-Z]+)(\:[0-9a-zA-Z]+)+', ''), 
            #-----
            (r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', ''), 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ],
    }
    #-------------------------
    return patterns_to_replace_by_typeid

In [ ]:
def get_std_patterns_to_replace_by_typeid():
    r"""
    Returns the standard patterns_to_replace_by_typeid.
    Function exists essentially to keep reduce_end_event_reasons_in_df_v2 from being too cluttered
    
    NOTE: Multi-line strings have more parentheses than absolutely necessary for clarity
    """
    #-------------------------
    patterns_to_replace_by_typeid = {
        # The following will be run at the beginning for all
        # 0th element: for removing MAC-esque meter codes
        'all_beg':          [
            #-----
            r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+'
            #-----
        ], 
        
        # The following will be run at the end for all
        # 0th element:
        #   The pattern removes any of the characters between [] (i.e., !"#$%&'(*+,-./:;<=>?@[\^_`{|~), 
        #     as many times as they show up, but ONLY at the end of the string
        # 1st element:
        #   Replace multiple spaces with single space
        'all_end':          [
            #-----
            r'[!"#$%&\'(*+,-./:;<=>?@[\\^_`{|~\s]*$', 
            #-----
            (r'\s{2,}', ' '), 
            #-----
            (r'\s*(.*)', r'\1')
            #-----
        ], 

        # addtnl regex = 1
        # total regex = 5
        '3.11.63.161':  [
            #-----
            (
                (
                    r'(Requested operation .* could not be applied to the given device type and firmware version.) '\
                    r'Device, DeviceType: .*, Firmware Version: .*$'
                ), 
                r'\1'
            )
            #-----
        ],

        # addtnl regex = 1
        # total regex = 5
        '3.12.136.38':  [
            #-----
            r'N/A'
            #-----
        ],

        # addtnl regex = 2
        # total regex = 6
        '3.12.136.85':  [
            #-----
            (r'(Secure association operation failed consecutively for 1 times for [0-9a-zA-Z]{4}.) .*', r'\1'), 
            #-----
            r'N/A'
            #-----
        ],

        # addtnl regex = 0
        # total regex = 4
        '3.12.17.257':  [],

        # addtnl regex = 1
        # total regex = 5
        '3.12.48.219':  [
            #-----
            (r'(Angle out of tolerance) \[.*\]', r'\1')
            #-----
        ],

        # addtnl regex = 0
        # total regex = 4
        '3.12.48.28':   [],

        # addtnl regex = 0
        # total regex = 4
        '3.12.93.219':  [],

        # addtnl regex = 0
        # total regex = 4
        '3.12.93.28':   [],

        # addtnl regex = 0
        # total regex = 4
        '3.18.72.28':   [],

        # addtnl regex = 0
        # total regex = 4
        '3.18.72.79':   [],

        # addtnl regex = 0
        # total regex = 4
        '3.21.3.79':    [],

        # addtnl regex = 1
        # total regex = 5
        '3.21.38.223':  [
            #-----
            (r'(Detected end of voltage sag on meter).*', r'\1')
            #-----
        ],

        # addtnl regex = 1
        # total regex = 5
        '3.21.38.248':  [
            #-----
            (r'(Detected end of voltage swell on meter).*', r'\1')
            #-----
        ],

        # addtnl regex = 0
        # total regex = 4
        '3.23.136.47':  [],

        # addtnl regex = 1
        # total regex = 5
        '3.23.136.85':  [
            #-----
            (r'Device Time: .* Failed Device Reason: (.*) Reboot Counter: .* Refresh Counter: .*', r'\1')
            #-----
        ],

        # addtnl regex = 0
        # total regex = 4
        '3.23.17.139':  [],

        # addtnl regex = 0
        # total regex = 4
        '3.26.0.216':   [],

        # addtnl regex = 0
        # total regex = 4
        '3.26.0.47':    [],

        # addtnl regex = 1
        # total regex = 5
        '3.26.136.216': [
            #-----
            (r'(NIC Power Restore Trap Received from device).*', r'\1')
            #-----
        ],

        # addtnl regex = 1
        # total regex = 5
        '3.26.136.47':  [
            #-----
            (
                (
                    r'(Last Gasp\s*-\s*[0-9a-zA-Z\s]*)[\s\:,.]*'\
                    r'.*'\
                    r'(Fail Reason: .*)$'
                ), 
                r'\1, \2'
            )
            #-----
        ],

        # addtnl regex = 1
        # total regex = 5
        '3.26.136.66':  [
            #-----
            (r'Device, (Last Gasp State: .*), (Detector State: .*), Reboot Count: \d*', r'\1, \2')
            #-----
        ],

        # addtnl regex = 2
        # total regex = 6
        '3.26.38.150':  [
            #-----
            (
                (
                    r'(Under Voltage)\s*'      
                    r'(\([0-9a-zA-Z\s]*\))\s*'\
                    r'([0-9a-zA-Z]*)?\s?'\
                    r'(for meter\:?\s*)'\
                    r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
                    r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
                    r'(Voltage out of tolerance)?'
                ), 
                r'\1 \2 \3 \4 \5'
            ), 
            #-----
            (
                (
                    r'(Under Voltage)\s*'
                    r'([0-9a-zA-Z]*)?\s*'\
                    r'(\([0-9a-zA-Z\s]*\))\s*'\
                    r'(for meter\:?\s*)'\
                    r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
                    r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
                    r'(Voltage out of tolerance)?'
                ), 
                r'\1 \3 \2 \4 \5'
            )
            #-----
        ],

        # addtnl regex = 4
        # total regex = 8
        '3.26.38.37':   [
            #-----
            (
                (
                    r'(Under Voltage)\s*'      
                    r'(\([0-9a-zA-Z\s]*\))\s*'\
                    r'([0-9a-zA-Z]*)?\s?'\
                    r'(for meter\:?\s*)'\
                    r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
                    r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
                    r'(Voltage out of tolerance)?'
                ), 
                r'\1 \2 \3 \4 \5'
            ), 
            #-----
            (
                (
                    r'(Under Voltage)\s*'
                    r'([0-9a-zA-Z]*)?\s*'\
                    r'(\([0-9a-zA-Z\s]*\))\s*'\
                    r'(for meter\:?\s*)'\
                    r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
                    r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
                    r'(Voltage out of tolerance)?'
                ), 
                r'\1 \3 \2 \4 \5'
            ), 
            #-----
            ('meterN/A', 'meter'), 
            #-----
            r'N/A'
            #-----
        ],

        # addtnl regex = 0
        # total regex = 4
        '3.26.38.47':   [],

        # addtnl regex = 1
        # total regex = 5
        '3.26.38.73':   [
            #-----
            ('meterN/A', 'meter')
            #-----
        ],

        # addtnl regex = 0
        # total regex = 4
        '3.26.38.93':   [],

        # addtnl regex = 0
        # total regex = 4
        '3.35.0.28':    [],

        # addtnl regex = 0
        # total regex = 4
        '3.35.0.40':    [],

        # addtnl regex = 1
        # total regex = 5
        '3.36.114.159': [
            #-----
            (r'(Ignoring (?:Interval|Register) Read data for device as it has time in the future) .*', r'\1')
            #-----
        ],

        # addtnl regex = 0
        # total regex = 4
        '3.36.114.73':  [],

        # addtnl regex = 1
        # total regex = 5
        '3.36.136.73':  [
            #-----
            (r'(Meter needs explicit time sync.) Drift: -?\d* s, (Encountered Problems:\s*.*), Meter_Time', r'\1 \2')
            #-----
        ],

        # addtnl regex = 0
        # total regex = 4
        '3.36.136.79':  [],

        # addtnl regex = 0
        # total regex = 4
        '3.7.19.242':   [],

        # addtnl regex = 0
        # total regex = 4
        '3.7.19.243':   [],

        # addtnl regex = 0
        # total regex = 4
        '3.8.0.215':    [],

        # addtnl regex = 1
        # total regex = 5
        '3.9.83.159':   [
            #-----
            (r'(Meter Program Seal mismatch for Device) \[Device ID, MAC Id\] = .*', r'\1')
            #-----
        ], 
        
        # addtnl regex = 3
        # total regex = 7
        '3.18.72.85':   [
            #-----
            (r'(Meter event Nonvolatile Memory Failure Detected)\s*Time event occurred on meter = .* Sequence number = .* User id = .*  Event argument = .*', r'\1'), 
            #-----
            (r'(Meter event Nonvolatile Memory Failure Detected).*', r'\1'),
            #-----
            (r'(Meter detected a nonvolatile memory failure).*', 'Meter event Nonvolatile Memory Failure Detected') 
            #-----
        ], 
        
        # addtnl regex = 0
        # total regex = 4
        '3.21.17.28':    [],
    }
    #-------------------------
    return patterns_to_replace_by_typeid

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Checked with over 15 million events from 2020, 2021, 2022
# Found new methods to be consistent with original
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
def reduce_end_event_reasons_in_df_v2(
    df, 
    reason_col='reason', 
    edetypeid_col='enddeviceeventtypeid', 
    patterns_to_replace_by_typeid=None, 
    addtnl_patterns_to_replace=None, 
    placement_col=None, 
    count=0, 
    flags=re.IGNORECASE,  
    inplace=True
):
    r"""
    Searches for each of patterns (= patterns_to_replace + addtnl_patterns_to_replace) in reason, if found, replaced in string.
      NOTE: Each pattern can be a single string, or a pair of strings.
      If a single string:
        the string represents the pattern and if found it is replaced by ''
      If a pair of strings:
        the first element is the pattern and the second element is the replacement
        NOTE: The replacement can be a string, a pattern, or a function!

    e.g., with default patterns:
        i.  
          'Under Voltage (CA000400) cleared for meter 00:13:50:01:00:a7:bc:52.'
    ==>   'Under Voltage (CA000400) cleared'
        ii.  
          'Last Gasp - NIC power lost for device: 00:13:50:02:00:9c:8f:2b, Reboot Count: 194, NIC timestamp: Jan 1, 2018 - 
           02:14:19 AM EST, Received timestamp: Jan 1, 2018 - 02:14:27 AM EST, Fail Reason: NIC power fail'
    ==>   'Last Gasp - NIC power lost'
        iii.  
          'Under Voltage (CA000400) cleared for meter 00:13:50:01:00:a7:bc:52. for device 00:13:50:01:00:a7:bc:52.'
    ==>   'Under Voltage (CA000400) cleared'

    ---------------------------------------------
    Description/motivation for default patterns:
    ---------------------------------------------
    - r'\:?\s?([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+'
      Matches all MAC-esque meter/device IDs
      NOTE: + used at end because want ONE OR MORE repetitions of (\:[0-9a-zA-Z]{1,2})
            * matches ZERO OR MORE 
            EXAMPLE: Assume strng = 'Demand Reset occurred for meter 00:13:50:01:00:17:c6:41.'
              If * used: match = 'De'
              If + used: match = ' 00:13:50:01:00:17:c6:41'
      EX:
            'Under Voltage (CA000400) cleared for meter 00:13:50:01:00:a7:bc:52.'
        ==> 'Under Voltage (CA000400) cleared for meter.'
    """
    #-------------------------
    if not inplace:
        df = df.copy()
    #-------------------------
    if patterns_to_replace_by_typeid is None:
        patterns_to_replace_by_typeid = get_std_patterns_to_replace_by_typeid()
    #-------------------------
    if addtnl_patterns_to_replace is not None:
        assert(isinstance(addtnl_patterns_to_replace, dict))
        patterns_to_replace_by_typeid = Utilities.supplement_dict_with_default_values(
            to_supplmnt_dict=patterns_to_replace_by_typeid, 
            default_values_dict=addtnl_patterns_to_replace,
            extend_any_lists=True
        )
    #-------------------------
    if placement_col is None:
        placement_col = reason_col
    #-------------------------
    # If 'all_beg' in patterns_to_replace_by_typeid, prepend contents to all
    # If 'all_end' in patterns_to_replace_by_typeid, append contents to all
    keys_to_skip_add = ['all_beg', 'all_end']
    #-----
    if 'all_beg' in patterns_to_replace_by_typeid.keys():
        patterns_for_all = patterns_to_replace_by_typeid.pop('all_beg')
        for edetypeid, patterns_to_replace in patterns_to_replace_by_typeid.items():
            if edetypeid in keys_to_skip_add:
                continue
            patterns_to_replace_by_typeid[edetypeid] = patterns_for_all + patterns_to_replace
    #-----
    if 'all_end' in patterns_to_replace_by_typeid.keys():
        patterns_for_all = patterns_to_replace_by_typeid.pop('all_end')
        for edetypeid, patterns_to_replace in patterns_to_replace_by_typeid.items():
            if edetypeid in keys_to_skip_add:
                continue
            patterns_to_replace_by_typeid[edetypeid] = patterns_to_replace + patterns_for_all
    #-------------------------
    # This would be handled in reduce_end_event_reason if not here, but doing it once here
    # should save time, especially for large DFs, I believe.
    for edetypeid, patterns_to_replace in patterns_to_replace_by_typeid.items():
        for i in range(len(patterns_to_replace)):
            pattern = patterns_to_replace[i]
            if Utilities.is_object_one_of_types(pattern, [list, tuple]):
                assert(len(pattern)==2)
                continue
            else:
                assert(isinstance(pattern, str))
                patterns_to_replace_by_typeid[edetypeid][i] = (pattern, '')
    #-------------------------
    df[placement_col] = df.apply(lambda x: AMIEndEvents.reduce_end_event_reason(x[reason_col], patterns_to_replace_by_typeid[x[edetypeid_col]]), axis=1)
    #-------------------------
    return df


def reduce_end_event_reasons_in_df_v3(
    df, 
    reason_col='reason', 
    edetypeid_col='enddeviceeventtypeid', 
    patterns_to_replace_by_typeid=None, 
    addtnl_patterns_to_replace=None, 
    placement_col=None, 
    count=0, 
    flags=re.IGNORECASE,  
    inplace=True
):
    r"""
    Searches for each of patterns (= patterns_to_replace + addtnl_patterns_to_replace) in reason, if found, replaced in string.
      NOTE: Each pattern can be a single string, or a pair of strings.
      If a single string:
        the string represents the pattern and if found it is replaced by ''
      If a pair of strings:
        the first element is the pattern and the second element is the replacement
        NOTE: The replacement can be a string, a pattern, or a function!

    e.g., with default patterns:
        i.  
          'Under Voltage (CA000400) cleared for meter 00:13:50:01:00:a7:bc:52.'
    ==>   'Under Voltage (CA000400) cleared'
        ii.  
          'Last Gasp - NIC power lost for device: 00:13:50:02:00:9c:8f:2b, Reboot Count: 194, NIC timestamp: Jan 1, 2018 - 
           02:14:19 AM EST, Received timestamp: Jan 1, 2018 - 02:14:27 AM EST, Fail Reason: NIC power fail'
    ==>   'Last Gasp - NIC power lost'
        iii.  
          'Under Voltage (CA000400) cleared for meter 00:13:50:01:00:a7:bc:52. for device 00:13:50:01:00:a7:bc:52.'
    ==>   'Under Voltage (CA000400) cleared'

    ---------------------------------------------
    Description/motivation for default patterns:
    ---------------------------------------------
    - r'\:?\s?([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+'
      Matches all MAC-esque meter/device IDs
      NOTE: + used at end because want ONE OR MORE repetitions of (\:[0-9a-zA-Z]{1,2})
            * matches ZERO OR MORE 
            EXAMPLE: Assume strng = 'Demand Reset occurred for meter 00:13:50:01:00:17:c6:41.'
              If * used: match = 'De'
              If + used: match = ' 00:13:50:01:00:17:c6:41'
      EX:
            'Under Voltage (CA000400) cleared for meter 00:13:50:01:00:a7:bc:52.'
        ==> 'Under Voltage (CA000400) cleared for meter.'
    """
    #-------------------------
    if not inplace:
        df = df.copy()
    #-------------------------
    if patterns_to_replace_by_typeid is None:
        patterns_to_replace_by_typeid = get_std_patterns_to_replace_by_typeid_EXPLICIT()
    #-------------------------
    if addtnl_patterns_to_replace is not None:
        assert(isinstance(addtnl_patterns_to_replace, dict))
        patterns_to_replace_by_typeid = Utilities.supplement_dict_with_default_values(
            to_supplmnt_dict=patterns_to_replace_by_typeid, 
            default_values_dict=addtnl_patterns_to_replace,
            extend_any_lists=True
        )
    #-------------------------
    if placement_col is None:
        placement_col = reason_col
    #-------------------------
    # If 'all_beg' in patterns_to_replace_by_typeid, prepend contents to all
    # If 'all_end' in patterns_to_replace_by_typeid, append contents to all
    keys_to_skip_add = ['all_beg', 'all_end']
    #-----
    if 'all_beg' in patterns_to_replace_by_typeid.keys():
        patterns_for_all = patterns_to_replace_by_typeid.pop('all_beg')
        for edetypeid, patterns_to_replace in patterns_to_replace_by_typeid.items():
            if edetypeid in keys_to_skip_add:
                continue
            patterns_to_replace_by_typeid[edetypeid] = patterns_for_all + patterns_to_replace
    #-----
    if 'all_end' in patterns_to_replace_by_typeid.keys():
        patterns_for_all = patterns_to_replace_by_typeid.pop('all_end')
        for edetypeid, patterns_to_replace in patterns_to_replace_by_typeid.items():
            if edetypeid in keys_to_skip_add:
                continue
            patterns_to_replace_by_typeid[edetypeid] = patterns_to_replace + patterns_for_all
    #-------------------------
    # This would be handled in reduce_end_event_reason if not here, but doing it once here
    # should save time, especially for large DFs, I believe.
    for edetypeid, patterns_to_replace in patterns_to_replace_by_typeid.items():
        for i in range(len(patterns_to_replace)):
            pattern = patterns_to_replace[i]
            if Utilities.is_object_one_of_types(pattern, [list, tuple]):
                assert(len(pattern)==2)
                continue
            else:
                assert(isinstance(pattern, str))
                patterns_to_replace_by_typeid[edetypeid][i] = (pattern, '')
    #-------------------------
    df[placement_col] = df.apply(lambda x: AMIEndEvents.reduce_end_event_reason(x[reason_col], patterns_to_replace_by_typeid[x[edetypeid_col]]), axis=1)
    #-------------------------
    return df

In [ ]:
patterns_to_replace_by_typeid_EXPLICIT = get_std_patterns_to_replace_by_typeid_EXPLICIT()
#--------------------
patterns_to_replace_by_typeid = get_std_patterns_to_replace_by_typeid()
#-----
# If 'all_beg' in patterns_to_replace_by_typeid, prepend contents to all
# If 'all_end' in patterns_to_replace_by_typeid, append contents to all
keys_to_skip_add = ['all_beg', 'all_end']
#-----
if 'all_beg' in patterns_to_replace_by_typeid.keys():
    patterns_for_all = patterns_to_replace_by_typeid.pop('all_beg')
    for edetypeid, patterns_to_replace in patterns_to_replace_by_typeid.items():
        if edetypeid in keys_to_skip_add:
            continue
        patterns_to_replace_by_typeid[edetypeid] = patterns_for_all + patterns_to_replace
#-----
if 'all_end' in patterns_to_replace_by_typeid.keys():
    patterns_for_all = patterns_to_replace_by_typeid.pop('all_end')
    for edetypeid, patterns_to_replace in patterns_to_replace_by_typeid.items():
        if edetypeid in keys_to_skip_add:
            continue
        patterns_to_replace_by_typeid[edetypeid] = patterns_to_replace + patterns_for_all
#-------------------------
# This would be handled in reduce_end_event_reason if not here, but doing it once here
# should save time, especially for large DFs, I believe.
for edetypeid, patterns_to_replace in patterns_to_replace_by_typeid.items():
    for i in range(len(patterns_to_replace)):
        pattern = patterns_to_replace[i]
        if Utilities.is_object_one_of_types(pattern, [list, tuple]):
            assert(len(pattern)==2)
            continue
        else:
            assert(isinstance(pattern, str))
            patterns_to_replace_by_typeid[edetypeid][i] = (pattern, '')
#--------------------
print(patterns_to_replace_by_typeid_EXPLICIT==patterns_to_replace_by_typeid)

In [ ]:
ede_df_1 = ede_df_OG.copy()
ede_df_2 = ede_df_OG.copy()

In [ ]:
start = time.time()
ede_df_1 = AMIEndEvents.reduce_end_event_reasons_in_df(ede_df_1, placement_col='reason_red')
print(time.time()-start)

In [ ]:
start = time.time()
ede_df_2 = reduce_end_event_reasons_in_df_v2(ede_df_2, placement_col='reason_red')
print(time.time()-start)

In [ ]:
print(ede_df_1.shape==ede_df_2.shape)
print(ede_df_1.equals(ede_df_2))
print(ede_df_1.shape[0])

In [ ]:
# ede_typeid_to_reason_paths = [
#     r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20200101_20201231\rcpo_dfs_GRP_BY_NO_OUTG_AND_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_no_outg_df_OG.pkl', 
#     r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20200101_20201231\rcpo_dfs_GRP_BY_OUTG_AND_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_df_OG.pkl', 
    
#     r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20210101_20211231\rcpo_dfs_GRP_BY_NO_OUTG_AND_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_no_outg_df_OG.pkl', 
#     r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20210101_20211231\rcpo_dfs_GRP_BY_OUTG_AND_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_df_OG.pkl',
    
#     r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20220101_20220930\rcpo_dfs_GRP_BY_NO_OUTG_AND_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_no_outg_df_OG.pkl', 
#     r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20220101_20220930\rcpo_dfs_GRP_BY_OUTG_AND_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_df_OG.pkl', 
    
#     r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221014\20180101_20181231\rcpo_dfs_GRP_BY_OUTG_AND_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_df_OG.pkl', 
    
#     r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221014\20190101_20191231\rcpo_dfs_GRP_BY_OUTG_AND_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_df_OG.pkl', 
#     r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221014\20190101_20191231\rcpo_dfs_GRP_BY_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_no_outg_df_OG.pkl',
    
#     r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221014\20200101_20201231\rcpo_dfs_GRP_BY_OUTG_AND_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_df_OG.pkl', 
#     r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221014\20200101_20201231\rcpo_dfs_GRP_BY_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_no_outg_df_OG.pkl', 
    
#     r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221014\20210101_20211231\rcpo_dfs_GRP_BY_OUTG_AND_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_df_OG.pkl', 
#     r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221014\20210101_20211231\rcpo_dfs_GRP_BY_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_no_outg_df_OG.pkl', 
    
#     r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221014\20220101_20220930\rcpo_dfs_GRP_BY_OUTG_AND_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_df_OG.pkl', 
#     r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221014\20220101_20220930\rcpo_dfs_GRP_BY_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_no_outg_df_OG.pkl'
# ]


ede_typeid_to_reason_paths = [
    r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20200101_20201231\rcpo_dfs_GRP_BY_NO_OUTG_AND_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_no_outg_df_OG.pkl', 
    r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20200101_20201231\rcpo_dfs_GRP_BY_OUTG_AND_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_df_OG.pkl', 
    
    r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20210101_20211231\rcpo_dfs_GRP_BY_NO_OUTG_AND_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_no_outg_df_OG.pkl', 
    r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20210101_20211231\rcpo_dfs_GRP_BY_OUTG_AND_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_df_OG.pkl',
    
    r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20220101_20220930\rcpo_dfs_GRP_BY_NO_OUTG_AND_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_no_outg_df_OG.pkl', 
    r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20220101_20220930\rcpo_dfs_GRP_BY_OUTG_AND_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_df_OG.pkl', 
    
    r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20230301\20220101_20221231\NoOutgs_Pristine\rcpo_dfs_GRP_BY_NO_OUTG_AND_XFMR\outg_td_window_1_to_30_days\ede_typeid_to_reason_no_outg_prstn_df_OG.pkl'
]

In [ ]:
for i,path in enumerate(ede_typeid_to_reason_paths):
    assert(os.path.exists(path))
    ede_typeid_to_reason_df_i = pd.read_pickle(path)
    if i==0:
        ede_typeid_to_reason_df = ede_typeid_to_reason_df_i.copy()
    else:
        ede_typeid_to_reason_df = AMIEndEvents.combine_two_ede_typeid_to_reason_dfs(
            ede_typeid_to_reason_df1=ede_typeid_to_reason_df, 
            ede_typeid_to_reason_df2=ede_typeid_to_reason_df_i,
            sort=True
        )

In [ ]:
ede_typeid_to_reason_df['n_reasons_in_id'] = ede_typeid_to_reason_df['reason'].apply(lambda x: len(x))
ede_typeid_to_reason_df

In [ ]:
max_len = 125
for idx, row in ede_typeid_to_reason_df.sort_index().iterrows():
    print(idx)
    print(f"\tn_reasons = {len(row['reason'])}")
    for reason_i in row['reason']:
        if len(reason_i) <= max_len:
            print(f"\t{reason_i}")
        else:
            n_split=0
            while(len(reason_i)>max_len):
                split_pos = reason_i.rfind(' ', 0, max_len)
                if n_split==0:
                    print(f"\t{reason_i[:split_pos]}")
                else:
                    print(f"\t\t{reason_i[:split_pos]}")
                reason_i = reason_i[split_pos:]
                n_split+=1
            # Unless reason_i is exactly divisible by max_len, there will be some residual
            #   leftover in reason_i that will not be caught by the while loop, as 
            #   len(reason_i) will be less than max_len
            if len(reason_i)>0:
                print(f"\t\t{reason_i[:split_pos]}")
    print()

In [ ]:
with open(r'C:\Users\s346557\Downloads\ede_typeid_to_reason.txt', 'w') as f:
    max_len = 1000
    for idx, row in ede_typeid_to_reason_df.sort_index().iterrows():
        f.write(f"{idx}\n")
        f.write(f"\tn_reasons = {len(row['reason'])}\n")
        for reason_i in row['reason']:
            if len(reason_i) <= max_len:
                f.write(f"\t{reason_i}\n")
            else:
                n_split=0
                while(len(reason_i)>max_len):
                    split_pos = reason_i.rfind(' ', 0, max_len)
                    if n_split==0:
                        f.write(f"\t{reason_i[:split_pos]}\n")
                    else:
                        f.write(f"\t\t{reason_i[:split_pos]}\n")
                    reason_i = reason_i[split_pos:]
                    n_split+=1
                # Unless reason_i is exactly divisible by max_len, there will be some residual
                #   leftover in reason_i that will not be caught by the while loop, as 
                #   len(reason_i) will be less than max_len
                if len(reason_i)>0:
                    f.write(f"\t\t{reason_i[:split_pos]}\n")
        f.write('\n')

# RESULTS FROM ERIC

In [ ]:
path = r'C:\Users\s346557\Downloads\run-1678729719896-part-r-00018.csv'
# end_events_df = pd.read_csv(path)
end_events_df = pd.read_csv(path, on_bad_lines='skip')

# end_events_df = df.copy()

In [ ]:
print(end_events_df.shape)
end_events_df.head()

In [ ]:
end_events_df.columns

In [ ]:
cols_of_interest_end_dev_event = TableInfos.AMIEndEvents_TI.std_columns_of_interest
cols_of_interest_met_prem      = TableInfos.MeterPremise_TI.std_columns_of_interest

patterns_to_replace=[
    r'\:?\s*([0-9a-zA-Z]{1,2})(\:[0-9a-zA-Z]{1,2})+', 
    
    (
        (
            r'(Under Voltage)\s*'\
            r'([0-9a-zA-Z]*)?\s*'\
            r'(\([0-9a-zA-Z\s]*\))\s*'\
            r'([0-9a-zA-Z]*)?\s?'\
            r'(for meter\:?\s*)'\
            r'(?:(?:[0-9a-zA-Z]{1,2})(?:\:[0-9a-zA-Z]{1,2})+)?[\s:,.]*'\
            r'(?:Phase\s{1,2}[ABC](?:(?:\s*and\s*[ABC])|(?:,\s*[ABC])*))?\s*'\
            r'(Voltage out of tolerance)?'
        ), 
        'AMIEndEvents.under_voltage_match_func'
    ), 
    
    (
        (
            r'(Last Gasp\s*-\s*[0-9a-zA-Z\s]*)[\s\:,.]*'\
            r'.*'\
            r'(Fail Reason: .*)$'
        ), 
        'AMIEndEvents.last_gasp_reduce_func'
    ), 
    
    (r'(Angle out of tolerance) \[.*\]', r'\1'), 
    
    (r'(NIC Power Restore Trap Received from device).*', r'\1'), #TODO This has much info, like last gasp
    
    (
        (
            r'(Requested operation .* could not be applied to the given device type and firmware version.) '\
            r'Device, DeviceType: .*, Firmware Version: .*$'
        ), 
        r'\1'
    ), 
    
    ('meterN/A', 'meter'),
    
    (r'(Meter needs explicit time sync.) Drift: -?\d* s, (Encountered Problems:\s*.*), Meter_Time', r'\1 \2'), 
    
    (r'(Meter Program Seal mismatch for Device) \[Device ID, MAC Id\] = .*', r'\1'), 
    
    (r'Device Time: .* Failed Device Reason: (.*) Reboot Counter: .* Refresh Counter: .*', r'\1'), 
    
    (r'(Ignoring (?:Interval|Register) Read data for device as it has time in the future) .*', r'\1'), 
    
    (r'(Secure association operation failed consecutively for 1 times for [0-9a-zA-Z]{4}.) .*', r'\1'), 
    
    (r'Device, (Last Gasp State: .*), (Detector State: .*), Reboot Count: \d*', r'\1, \2'), 
    
    (r'(Detected end of voltage sag on meter).*', r'\1'), 
    
    (r'(Detected end of voltage swell on meter).*', r'\1'), 
    
    r'N/A', 
    
    (r'\s{2,}', ' ')
]

In [ ]:
# Below, when running reduce_end_event_reasons_in_df, the default behavior (placement_col=None) is to simply replace
#   the entries in the 'reason' column by their reduced versions.
# For purposes here, I will keep both the full reason column (renamed to 'reason_full') and the reduced reason column (named 'reason')

print(f'end_events_df.shape[0]                          = {end_events_df.shape[0]}')
print(f"end_events_df['serialnumber'].nunique()         = {end_events_df['serialnumber'].nunique()}")
print(f"end_events_df['enddeviceeventtypeid'].nunique() = {end_events_df['enddeviceeventtypeid'].nunique()}")
print('\n\n')

end_events_df = AMIEndEvents.reduce_end_event_reasons_in_df(
    df=end_events_df, 
    patterns_to_replace=patterns_to_replace, 
    reason_col='reason', 
    placement_col='curated_reason_2a'
)

end_events_df = reduce_end_event_reasons_in_df_v2(
    df=end_events_df, 
    reason_col='reason', 
    placement_col='curated_reason_2b'
)

end_events_df = reduce_end_event_reasons_in_df_v3(
    df=end_events_df, 
    reason_col='reason', 
    placement_col='curated_reason_2c'
)

print('BEFORE REASON REDUCTION')
print(f"end_events_df['reason'].nunique()               = {end_events_df['reason'].nunique()}")
print('AFTER REASON REDUCTION')
print(f"end_events_df['curated_reason_2a'].nunique()     = {end_events_df['curated_reason_2a'].nunique()}")
print(f"end_events_df['curated_reason_2b'].nunique()     = {end_events_df['curated_reason_2b'].nunique()}")
print(f"end_events_df['curated_reason'].nunique()        = {end_events_df['curated_reason'].nunique()}")

In [ ]:
end_events_df['reason']

In [ ]:
end_events_df['curated_reason'] = end_events_df['curated_reason'].str.replace('"', '')

In [ ]:
end_events_df['curated_reason']

In [ ]:
print((end_events_df['curated_reason'] != end_events_df['curated_reason_2a']).sum())
print((end_events_df['curated_reason'] != end_events_df['curated_reason_2b']).sum())
print((end_events_df['curated_reason_2a'] != end_events_df['curated_reason_2b']).sum())
print((end_events_df['curated_reason_2a'] != end_events_df['curated_reason_2c']).sum())

In [ ]:
ede_df_neq = end_events_df[end_events_df['curated_reason'] != end_events_df['curated_reason_2a']].copy()
ede_df_neq

In [ ]:
print(end_events_df['enddeviceeventtypeid'].nunique())
print(ede_df_neq['enddeviceeventtypeid'].nunique())

In [ ]:
ede_df_neq['enddeviceeventtypeid'].unique()

In [ ]:
ede_df_neq.iloc[0]['reason']

In [ ]:
ede_df_neq.iloc[0]['curated_reason_2a']

In [ ]:
ede_df_neq[['enddeviceeventtypeid', 'reason', 'curated_reason', 'curated_reason_2a']]

In [ ]:
print(ede_df_neq.iloc[0]['reason'])
print(ede_df_neq.iloc[0]['curated_reason'])
print(ede_df_neq.iloc[0]['curated_reason_2a'])

In [ ]:
ede_df_neq_notvoltg = ede_df_neq[~ede_df_neq['enddeviceeventtypeid'].isin(['3.26.38.37', '3.26.38.150'])].copy()
ede_df_neq_notvoltg[['enddeviceeventtypeid', 'reason', 'curated_reason', 'curated_reason_2a']]

In [ ]:
print(ede_df_neq_notvoltg.iloc[0]['reason'])
print(ede_df_neq_notvoltg.iloc[0]['curated_reason'])
print(ede_df_neq_notvoltg.iloc[0]['curated_reason_2a'])

In [ ]:
print(ede_df_neq_notvoltg.loc[2686566]['reason'])
print(ede_df_neq_notvoltg.loc[2686566]['curated_reason'])
print(ede_df_neq_notvoltg.loc[2686566]['curated_reason_2a'])

In [ ]:
ede_df_neq.sort_values(by=[], ignore_index=True)

In [ ]:
ede_df_neq.loc[2687003]['reason']

In [ ]:
end_events_df[end_events_df['enddeviceeventtypeid']=='3.12.136.85']['reason']

In [ ]:
ede_df_neq[['curated_reason', 'curated_reason_2a']].drop_duplicates().index

In [ ]:
ede_df_neq.loc[ede_df_neq[['curated_reason', 'curated_reason_2a']].drop_duplicates().index][['enddeviceeventtypeid', 'reason', 'curated_reason', 'curated_reason_2a']]

In [ ]:
ede_df_neq_mwe = ede_df_neq.loc[ede_df_neq[['curated_reason', 'curated_reason_2a']].drop_duplicates().index].copy()
ede_df_neq_mwe

In [ ]:
for idx,row in ede_df_neq_mwe.iterrows():
    print(f"enddeviceeventtypeid   = {row['enddeviceeventtypeid']}")
    print(f"reason                 = {row['reason']}")
    print(f"curated_reason         = {row['curated_reason']}")
    print(f"curated_reason_correct = {row['curated_reason_2a']}")
    print()

In [ ]:
print(ede_df_neq.iloc[1]['reason'])
print(ede_df_neq.iloc[1]['curated_reason'])
print(ede_df_neq.iloc[1]['curated_reason_2a'])
print(ede_df_neq.iloc[1]['curated_reason_2b'])

In [ ]:
end_events_df[end_events_df['reason'].str.contains('Under Voltage')]['reason']

In [ ]:
rcpo_df = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20200101_20201231\rcpo_dfs_GRP_BY_OUTG_AND_XFMR\outg_td_window_1_to_30_days\rcpo_df_norm.pkl')

In [ ]:
reasons = sorted(rcpo_df.columns.unique().tolist())
reasons

In [ ]:
len(reasons)

In [ ]:
rcpo_df_no_outg = pd.read_pickle(r'C:\Users\s346557\Documents\LocalData\dovs_and_end_events_data\20221216\20200101_20201231\rcpo_dfs_GRP_BY_NO_OUTG_AND_XFMR\outg_td_window_1_to_30_days\rcpo_no_outg_df_norm.pkl')

In [ ]:
reasons_no_outg = sorted(rcpo_df_no_outg.columns.unique().tolist())
reasons_no_outg

In [ ]:
len(reasons_no_outg)

In [ ]:
import pyspark

# meter_events.events_summar Table

In [ ]:
conn_db = Utilities.get_athena_dev_aws_connection()

In [ ]:
# sql = r"""
# SELECT *
# FROM meter_events.events_summary
# WHERE aep_state='OH'
# AND   aep_event_dt BETWEEN '2022-01-01' AND '2022-01-07'
# """

# sql = r"""
# SELECT *
# FROM meter_events.events_summary
# LIMIT 100
# """

sql = r"""
SELECT *
FROM stg_meterevents.end_device_event_catg_stg
WHERE aep_event_dt BETWEEN '2023-01-01' AND '2023-01-07'
"""

In [ ]:
df = pd.read_sql_query(sql, conn_db) 

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
end_events_df = df.copy()
end_events_df

In [ ]:
# Below, when running reduce_end_event_reasons_in_df, the default behavior (placement_col=None) is to simply replace
#   the entries in the 'reason' column by their reduced versions.
# For purposes here, I will keep both the full reason column (renamed to 'reason_full') and the reduced reason column (named 'reason')

print(f'end_events_df.shape[0]                          = {end_events_df.shape[0]}')
print(f"end_events_df['serialnumber'].nunique()         = {end_events_df['serialnumber'].nunique()}")
print(f"end_events_df['enddeviceeventtypeid'].nunique() = {end_events_df['enddeviceeventtypeid'].nunique()}")
print('\n\n')

end_events_df = AMIEndEvents.reduce_end_event_reasons_in_df(
    df=end_events_df, 
    patterns_to_replace_by_typeid=None, 
    reason_col='reason', 
    placement_col='curated_reason_2a'
)

# end_events_df = reduce_end_event_reasons_in_df_v2(
#     df=end_events_df, 
#     reason_col='reason', 
#     placement_col='curated_reason_2b'
# )

# end_events_df = reduce_end_event_reasons_in_df_v3(
#     df=end_events_df, 
#     reason_col='reason', 
#     placement_col='curated_reason_2c'
# )

print('BEFORE REASON REDUCTION')
print(f"end_events_df['reason'].nunique()               = {end_events_df['reason'].nunique()}")
print('AFTER REASON REDUCTION')
print(f"end_events_df['curated_reason_2a'].nunique()     = {end_events_df['curated_reason_2a'].nunique()}")
# print(f"end_events_df['curated_reason_2b'].nunique()     = {end_events_df['curated_reason_2b'].nunique()}")
print(f"end_events_df['curated_reason'].nunique()        = {end_events_df['curated_reason'].nunique()}")

In [ ]:
print('BEFORE REASON REDUCTION')
print(f"end_events_df['reason'].nunique()               = {end_events_df['reason'].nunique()}")
print('AFTER REASON REDUCTION')
print(f"end_events_df['curated_reason_2a'].nunique()     = {end_events_df['curated_reason_2a'].nunique()}")
# print(f"end_events_df['curated_reason_2b'].nunique()     = {end_events_df['curated_reason_2b'].nunique()}")
print(f"end_events_df['curated_reason'].nunique()        = {end_events_df['curated_reason'].nunique()}")

In [ ]:
end_events_df['curated_reason_2a'].equals(end_events_df['curated_reason'])

In [ ]:
print(new_id.get_sql_statement())

In [ ]:
new_id = AMIEndEvents(
    df_construct_type=DFConstructType.kRunSqlQuery, 
    build_sql_function_kwargs=dict(
        cols_of_interest=TableInfos.AMIEndEvents_TI.std_columns_of_interest, 
        date_range=['2022-01-01', '2022-01-07'], 
        enddeviceeventtypeid=['3.25.17.3']
    ), 
    init_df_in_constructor=True
)

In [ ]:
new_id_df = new_id.df.copy()
new_id_df

In [ ]:
new_id_df['reason']

In [ ]:
for idx, row in new_id_df.iloc[:100].iterrows():
    print(row['reason'])

In [ ]:
distinct = AMIEndEvents.get_end_event_distinct_fields(
    '2020-01-01', 
    '2023-03-21', 
    fields=['enddeviceeventtypeid']
)

In [ ]:
distinct

In [ ]:
len(get_std_patterns_to_replace_by_typeid_EXPLICIT_OG().keys())